In [ ]:
import scanpy as sc
import besca as bc
import os
import bbknn
import numpy as np
import scvelo as scv
import pandas as pd
import cellrank as cr

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
from scipy import stats
import itertools


In [ ]:

color_dict = {'R': 'coral', 'TF': 'firebrick', 'NR_nadj': 'lightskyblue','NR_adj': 'royalblue'}

In [ ]:
species='human'
scv.settings.set_figure_params('scvelo')
#adata = scv.read(filename, cache=True)

# Reading the h5ad

In [ ]:
WD=''
analysis_name='sw_besca2_immune'
dcdata=sc.read('Fullanalysis/analyzed/sw_besca2_immune/sw_besca2_immune.annotated.filtered.h5ad')

In [ ]:

outpath='Fullanalysis/analyzed/sw_besca2_immune/velocity/'

In [ ]:
figdir=outpath


figdir=os.path.join(outpath+'/publication/')
sc.settings.figdir = figdir

scv.settings.figdir = figdir
if not os.path.exists(figdir):
    os.makedirs(figdir)

In [ ]:
velosubset='CD8Tcell'

In [ ]:
sc.set_figure_params()

In [ ]:
sc.pl.umap(dcdata, color=['leiden', 'leiden_TNK', 'Sample type'],legend_loc='on data', legend_fontsize=0.6)

In [ ]:
sc.set_figure_params()
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_rc_cells'], size=8)

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_merged'], size=8)

In [ ]:
sc.set_figure_params()
sc.pl.umap(dcdata,color=['CD8A','CD8B','CD4','CD3D'], color_map='viridis')

In [ ]:
### Subset to cells annotated as CD8s and remove NK-like T cells 

dcdata=dcdata[(dcdata.obs['celltype2_pub'].isin(['CD8-positive T cell']))].copy()
if (velosubset=='CD8Tcell'):
    dcdata=dcdata[(dcdata.obs['celltype3_pub']!='mature NK T cell')].copy()


In [ ]:
dcdata.obs['leidenf']=dcdata.obs['leiden'].copy()

In [ ]:
sc.pl.umap(dcdata, color=['leiden', 'celltype3_pub','celltype3', 'Sample type'])

In [ ]:
sc.pl.umap(dcdata, color=['leiden', 'celltype3_pub','celltype3', 'Sample type'])

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_pub'],legend_fontsize=8, 
           save='-subsetonoriginalmap-'+velosubset+'.pdf') #14, 34, 3, 31


In [ ]:
sc.pl.umap(dcdata, color=['celltype3_pub'],legend_fontsize=8, 
           save='-subsetonoriginalmap-'+velosubset+'.pdf') #14, 34, 3, 31


In [ ]:
sc.pl.umap(dcdata, color=['CD8A','CD8B','CD4'])

In [ ]:
#### Calculate neighbours using PatientID as a batch

bbknn.bbknn(dcdata, batch_key='PatientID', use_rep='X_pca')

In [ ]:
sc.tl.diffmap(dcdata)


sc.pp.neighbors(dcdata, n_neighbors=15, use_rep='X_diffmap')
sc.tl.leiden(dcdata, resolution=0.5)


sc.tl.paga(dcdata, groups='leiden')

sc.pl.umap(dcdata,color='leiden')


In [ ]:
sc.pl.paga(dcdata)


In [ ]:
sc.tl.umap(dcdata, init_pos='paga')

sc.pl.umap(dcdata,color=['leiden','leidenf'],legend_loc='on data')

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_pub'],legend_fontsize=8, 
          save='-subsettrajectorymap-'+velosubset+'.celltype3_pub.pdf') #14, 34, 3, 31


In [ ]:
sc.pl.umap(dcdata, color=['PatientID'],legend_fontsize=8,
          save='-subsettrajectorymap-'+velosubset+'.PatientID.pdf') #14, 34, 3, 31


In [ ]:
sc.pl.umap(dcdata, color=['Sample type'],legend_fontsize=8,
          save='-subsettrajectorymap-'+velosubset+'.SampleType.pdf') #14, 34, 3, 31


In [ ]:
#### read the already pooled loom file containing concatenated sample information (generated through velocyto)
loom=scv.read('Fullanalysis/loom_afterconcat.h5ad')

In [ ]:
#### merge and save an intermediate file 
adata=scv.utils.merge(dcdata, loom).copy()
#adata.write("Fullanalysis/CD8Tcells-loom.h5ad", compression="gzip")

In [ ]:
sc.set_figure_params(6)
sc.pl.umap(adata,color=['PatientID'],ncols=2)

### Breakdown of clusters per individuals 

In [ ]:
sc.pl.umap(adata,color=['RCat'],ncols=2,save='-subsettrajectorymap-'+velosubset+'.RCat.pdf')

In [ ]:
adata.write(WD+'/analyzed/'+analysis_name+'/'+analysis_name+'.annotated.'+velosubset+'.h5ad')

In [ ]:
adata=sc.read(WD+'/analyzed/'+analysis_name+'/'+analysis_name+'.annotated.'+velosubset+'.h5ad')

In [ ]:
### Remove clusters that are mainly patient-specific + solely proliferating
subdata=adata[adata.obs['leiden']!='2'].copy()
subdata=subdata[subdata.obs['leiden']!='4'].copy()
subdata=subdata[subdata.obs['leiden']!='14'].copy()
subdata=subdata[subdata.obs['leiden']!='12'].copy()

### subdata=subdata[subdata.obs.sample(frac=0.5).index].copy() # for speed/memory reasons, one could subset to 50% of data

In [ ]:
sc.pl.umap(subdata,color=['leiden'],legend_loc='on data')

In [ ]:
#sc.pl.umap(subdata,color=['PatientID'],legend_loc='on data')

In [ ]:
sc.pl.umap(adata,color=['leiden'],legend_loc='on data')

In [ ]:
sc.pl.umap(adata,color=['celltype3_pub'])

In [ ]:
### Run velocity analysis on the subsetted data 

scv.pp.filter_and_normalize(subdata)
scv.pp.moments(subdata)
scv.pp.neighbors(subdata)

scv.tl.recover_dynamics(subdata)
scv.tl.velocity(subdata,mode='dynamical')
scv.tl.velocity_graph(subdata)

In [ ]:
### Run velocity analysis on the entire set, for completion
scv.pp.filter_and_normalize(adata)
scv.pp.moments(adata)
scv.pp.neighbors(adata)

scv.tl.recover_dynamics(adata)
scv.tl.velocity(adata,mode='dynamical')
scv.tl.velocity_graph(adata)

In [ ]:
scv.utils.randomized_velocity(subdata)
#scv.pl.velocity_embedding_stream(adata, vkey=['velocity', 'velocity_random'])

In [ ]:
mysubset='All'
subcat='PBMCandTIL'
#subcat='TILonly'

In [ ]:
### Save and reread intermediate analysis results 
subdata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.subsampled.h5ad')
adata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')

adata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')
subdata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.subsampled.h5ad')

In [ ]:
scv.pl.proportions(adata, groupby='leiden')

In [ ]:
scv.pl.proportions(adata, groupby='experiment')

In [ ]:

#scv.pl.velocity(adata, ['NR4A2',  'ZNF331', 'CREM','PRDM1','LYST','PKM'], ncols=2,groupby='leiden')

In [ ]:
scv.tl.velocity_confidence(subdata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(subdata, c=keys, cmap='coolwarm', perc=[5, 95])


In [ ]:
scv.tl.velocity_confidence(subdata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(subdata, c=keys, cmap='coolwarm', perc=[5, 95])


In [ ]:
scv.tl.velocity_confidence(adata)


In [ ]:
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata, c=keys, cmap='coolwarm', perc=[5, 95],
               save='-subsettrajectorymap-'+velosubset+'.VeloLenConf.pdf')


In [ ]:
scv.pl.velocity_graph(subdata, threshold=.4, color='leiden',
                      save='-trajectorymap-'+velosubset+'.VeloGraph.pdf')

In [ ]:
scv.pl.velocity_graph(subdata, threshold=.2, color='leiden')

In [ ]:
scv.pl.velocity_graph(adata, threshold=.4, color='leiden',
                      save='-subsettrajectorymap-'+velosubset+'.VeloGraph.pdf')

In [ ]:
sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,8)
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['Sample type'],
                                 save='-trajectorymap-'+velosubset+'.Velocity-Sample_Type.pdf')

In [ ]:
scv.pl.velocity_embedding_stream(subdata, vkey=[ 'velocity_random'],color=['celltype3_pub'],legend_loc='right',
                                 save='-trajectorymap-'+velosubset+'.VelocityRandom-Celltype3.pdf')

In [ ]:
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['celltype3_pub'],legend_loc='right',
                                 save='-trajectorymap-'+velosubset+'.Velocity-Celltype3.pdf')

In [ ]:
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['leiden','Sample type'])

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['leiden','Sample type'],min_mass=3.4, 
                                save='-subsettrajectorymap-'+velosubset+'.Velocity.pdf')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['celltype3_pub'], 
                                 legend_loc='right',min_mass=3.4,
                                 save='-subsettrajectorymap-'+velosubset+'.Velocity_celltype3_pub.pdf')

In [ ]:
scv.pl.velocity_embedding_stream(adata[adata.obs.sample(frac=1).index], basis='umap',color=['RCat'], 
                                 legend_loc='right',min_mass=3.4,
                                 save='-subsettrajectorymap-'+velosubset+'.Velocity_RCat.pdf')

In [ ]:
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['celltype3_pub'])

In [ ]:
scv.tl.score_genes_cell_cycle(adata)
scv.pl.scatter(adata, color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95],
               save='CellCycle-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-CCPhase.png')
s_genes, g2m_genes = scv.utils.get_phase_marker_genes(adata)


In [ ]:
scv.pl.scatter(adata,color=['TCF7','IL7R','GZMK','GZMH','GZMB','PRF1',
                            'PDCD1','TOX','HAVCR2','GNLY','NKG7','XCL1'], ncols=3)

In [ ]:
scv.pl.scatter(adata, color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95],
               save='-subsettrajectorymap-'+velosubset+'.cell_cycle.pdf')


In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['Sample type'],
                                 min_mass=3.25, 
                                 save='-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-SampleType.png')

In [ ]:
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['Sample type'],
                                 min_mass=3.25, 
                                 save='-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-SampleType.subdata.png')

In [ ]:
sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:

scv.pl.velocity_embedding_stream(adata, basis='umap',color=['celltype3_merged'], 
                                 legend_loc='best', 
                                save='-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-celltype3.png') #min_mass=3.25,

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['leiden'], legend_loc='on data')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['leiden_TNK'], legend_loc='on data')

In [ ]:
sc.set_figure_params()

In [ ]:
sc.pl.umap(adata,color=['XCL1','CCR7','SELL','MKI67', 'STMN1','ENTPD1','CD38','TOX','CD4','CD8A','CD8B', 'CD3D'], 
           color_map='viridis',save='-subsettrajectorymap-'+velosubset+'.Markers1.pdf')

### Resource (stem-like) T cells markers

In [ ]:
sc.pl.umap(adata,color=['PDCD1','LAG3','CD28','HAVCR2', 'CXCR5','TCF7', 'CD8A','IL7R'], 
           color_map='viridis',save='-subsettrajectorymap-'+velosubset+'.Markers2.pdf')

In [ ]:
sc.pl.dotplot(adata, var_names=['PDCD1','LAG3','CD28','HAVCR2', 'CXCR5','TCF7', 'CD8A','IL7R', 'CD8A','CD8B'], 
              groupby='leiden', dot_max=0.2,vmax=1,save='-subsettrajectorymap-'+velosubset+'.Markers1.newleiden.pdf')

In [ ]:
sc.pl.dotplot(adata, var_names=['PDCD1','LAG3','CD28','HAVCR2', 'CXCR5','TCF7', 'CD8A','IL7R', 'CD8A','CD8B'], 
              groupby='celltype3_pub', dot_max=0.2,vmax=1,save='-subsettrajectorymap-'+velosubset+'.Markers1.celltype3_pub.pdf')

In [ ]:
#corder=['1','2','3','5','7','8','9','10']

In [ ]:
sc.pl.dotplot(adata, var_names=['PDCD1','LAG3','CD28','HAVCR2', 'CXCR5','TCF7', 'CD8A','IL7R', 'CD8A','CD8B'], 
              groupby='leidenf', dot_max=0.2,vmax=1)

In [ ]:
scv.pl.velocity_embedding(adata, dpi=120, arrow_size=2, arrow_length=2,color=['leiden','celltype3_merged'])

In [ ]:
sc.set_figure_params()

In [ ]:
sc.pl.umap(adata,color=['Sample type'])

In [ ]:
sc.pl.umap(adata,color=['leiden'],legend_loc='on data')

In [ ]:
sc.pl.umap(adata,color=['leidenf'],legend_loc='on data')

In [ ]:
clustercomp=bc.tl.count_occurrence_subset(adata, 'leiden', count_variable ='PatientID', return_percentage = True)


In [ ]:
clustercomp[clustercomp.max().sort_values().index].transpose()

In [ ]:
sc.set_figure_params()
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,5)
import matplotlib 

#clustercomp=clustercomp[clustercomp.max().sort_values().index].copy()
#clustercomp.transpose().plot(kind='bar', stacked=True)
clustercomp[clustercomp.max().sort_values().index].transpose().plot.bar(stacked=True, figsize=(12, 8))
plt.ylabel('Percentage donor per cluster')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#fig.savefig(figdir+'Clustercomp-'+velosubset+'.PatientID.pdf'),bbox_inches='tight')
plt.savefig(figdir+'Clustercomp-'+velosubset+'.PatientID.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
clustercomp=bc.tl.count_occurrence_subset(adata, 'leiden', count_variable ='RCat', return_percentage = True)


In [ ]:
clustercomp=clustercomp[clustercomp.max().sort_values().index].copy()
clustercomp.transpose().plot(kind='bar', stacked=True,color=[color_dict[i] for i in list(clustercomp.index)])
plt.ylabel('Percentage R per cluster')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#plt.savefig(figdir+'Clustercomp-'+velosubset+'.RCat.pdf',bbox_inches='tight')
plt.savefig(figdir+'Clustercomp-'+velosubset+'.RCat.svg', format='svg', bbox_inches="tight", dpi=300)

### Run cellrank analysis to estimate initial and terminal states 

In [ ]:
cr.settings.verbosity = 2
cr.tl.terminal_states(subdata, cluster_key='leiden', 
                      weight_connectivities=0.2)


In [ ]:
cr.pl.terminal_states(subdata)
cr.tl.initial_states(subdata, cluster_key='leiden') 
cr.pl.initial_states(subdata, discrete=True)

cr.tl.lineages(subdata)
cr.pl.lineages(subdata, same_plot=False)


In [ ]:
import matplotlib.pyplot as plt
plt.hist(subdata.obs['initial_states_probs'])


In [ ]:

plt.hist(subdata.obs['terminal_states_probs'])


In [ ]:
scv.tl.recover_latent_time(subdata, root_key='initial_states_probs', end_key='terminal_states_probs')


In [ ]:
scv.tl.paga(subdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


In [ ]:
sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:
cr.pl.cluster_fates(subdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=5, edge_width_scale=1, max_edge_width=4, title='directed PAGA')


In [ ]:
cr.tl.lineage_drivers(subdata, use_raw=False, backward=False)

In [ ]:
### Calculate lineage drivers, per clsuter
sc.pl.umap(subdata,color=['leiden'], legend_loc='on data')

In [ ]:
cr.tl.lineage_drivers(subdata, use_raw=True, backward=False,cluster_key='leiden', clusters=['8','9','0','11'])
lin_drivers_c8_9_0_11=subdata.raw.var['to 5']

cr.tl.lineage_drivers(subdata, use_raw=True, backward=False,cluster_key='leiden', clusters=['16','3','7','1'])
lin_drivers_c16_3_7_1=subdata.raw.var['to 5']

cr.tl.lineage_drivers(subdata, use_raw=True, backward=False,cluster_key='leiden', clusters=['15','13'])
lin_drivers_c15_13=subdata.raw.var['to 5']


cr.tl.lineage_drivers(subdata, use_raw=True, backward=False,cluster_key='leiden', clusters=['11','5','6'])
lin_drivers_c11_5_6=subdata.raw.var['to 5']


In [ ]:
sc.set_figure_params()
cr.tl.lineage_drivers(subdata)

In [ ]:
cr.pl.lineage_drivers(subdata, lineage="5", n_genes=10)

In [ ]:
oi=list(subdata.var['to 5'].sort_values(ascending=False)[0:50].index)+list(subdata.var['to 5'].sort_values()[0:50].index)

In [ ]:
mygenes=cr.pl.heatmap(subdata, model, 
              genes=oi,
              show_absorption_probabilities=True,use_raw=True
              ,show_all_genes=True,cluster_genes=True,
              lineages="5", n_jobs=4, backend='loky',figsize=(10,18),return_genes=True)

In [ ]:
cdata=adata.copy()

In [ ]:
#cdata.uns['iroot']=np.where(cdata.obs['CELL']==list(subdata[subdata.uns['iroot']].obs['CELL'])[0])[0][0]
scv.tl.latent_time(cdata,min_likelihood=0.2, min_confidence=0.9) # #,root_key='iroot' min_likelihood=0.2, min_confidence=0.9
#scv.tl.latent_time(adata)
scv.pl.scatter(cdata, color='latent_time', color_map='gnuplot', size=30,
               save='-subsettrajectorymap-'+velosubset+'.LatentTime.pdf')

In [ ]:
sc.pl.umap(cdata, color=['root_cells','end_points'],
           save='-subsettrajectorymap-'+velosubset+'.RootAndEnd.pdf')

In [ ]:
sc.pl.umap(adatanocc, color=['root_cells','end_points'])

In [ ]:
#### save and re-read files after the cellrank analysis part
#adata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')
#subdata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.subsampled.h5ad')
#adata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')
#subdata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.subsampled.h5ad')

#tdata.write("Fullanalysis/analyzed/sw_besca2_immune/velocity/Velo-CD8Tcell-All-PBMCandTIL-dyn_after_terminal_initial_exclude.h5ad")
#alldata.write("Fullanalysis/analyzed/sw_besca2_immune/velocity/Velo-CD8Tcell-All-PBMCandTIL-dyn_after_terminal_initial.h5ad")

adata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')
tdata=sc.read("Fullanalysis/analyzed/sw_besca2_immune/velocity/Velo-CD8Tcell-All-PBMCandTIL-dyn_after_terminal_initial_exclude.h5ad")
alldata=sc.read("Fullanalysis/analyzed/sw_besca2_immune/velocity/Velo-CD8Tcell-All-PBMCandTIL-dyn_after_terminal_initial.h5ad")

In [ ]:
sc.set_figure_params()

In [ ]:
scv.pl.scatter(alldata, color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95], 
               save='CellCycle'+velosubset+'-'+mysubset+'-'+subcat+'.svg')


In [ ]:
scv.pl.velocity_embedding_stream(alldata, basis='umap',color=['Sample type', 'leiden','celltype3_pub'],
                                 min_mass=3.25,save='Fulldata-'+velosubset+'-'+mysubset+'-'+subcat+'.svg')

In [ ]:
xdata=tdata.copy()
scv.pp.neighbors(xdata)

scv.utils.randomized_velocity(xdata)
#scv.pl.velocity_embedding_stream(adata, vkey=['velocity', 'velocity_random'])

In [ ]:
scv.pl.velocity_embedding_stream(xdata, vkey=['velocity_random'], color='Sample type')

In [ ]:
sc.set_figure_params()

In [ ]:
#set(tdata.obs.columns)

In [ ]:
sc.pl.umap(tdata,color=['leiden'])

In [ ]:
sc.pl.umap(tdata,color=['leiden'], legend_loc='on data',save='Finalumap-'+velosubset+'.leiden.tdata.svg')

In [ ]:
sc.pl.umap(tdata,color=['celltype3_pub'],save='Finalumap-'+velosubset+'.celltype3_pub.tdata.svg')

### Plot distinct sets of genes of interest 

In [ ]:
goi=['KLRC2', 'CXCR6', 'CXCL13', 'HAVCR2', 'ITGA1', 'FOS',
     'CD69', 'KLRF1', 'ZNF683', 'KLRG1',
      'HSPA1A', 'HSPA1B', 'ATF3', 'DUSP4', 'TIGIT',
     'CCL4', 'GZMB','PRF1', 'XCL1', 'XCL2', 'CAV1', 'CRTAM',
     'CD200', 'FCRL6', 'MATK', 'GZMA', 'HLA-DQA1', 'ACTN1', 
      'ENTPD1', 'DUSP1', 'IL18R1','CXCR5','CXCR3','SLAMF6',
     'CTLA4', 'PRDM1', 'TYMS', 'MCM10', 'CDC45', 'ZWINT', 'CLSPN',
     'CDC20', 'DLGAP5', 'BIRC5', 'AURKB', 'CDCA2',
    'STMN1', 'MKI67', 'TCF7', 'PLAC8', 'GZMK', 'IFNG', 'IL2', 'PDCD1',
     'LAG3', 'TOX', 'ITGAE', 'CD28', 
     'CD27', 'TNFRSF9', 'CD101', 'CD38', 'PIK3R1']

In [ ]:
goi=['CDCA2','CDC20','DLGAP5','MKI67','BIRC5','AURKB','TYMS','STMN1',
     'CD40LG','TMIGD2','ACTN1','KLF2','PLAC8','TCF7','SELL','CCR7','IL7R',  'PTGER2',
      'PIK3R1','KLRG1','ZNF683','FCRL6','MATK','GNLY',
     'XCL1','XCL2','CD200','CXCR3','SLAMF6','IL2','IL18R1','CXCR5','ID3','S1PR1','KLF3','IFNG','CCL4','CCL3','CXCL13',
     'KLRF1','KLRB1','CRTAM','CD28','CAV1','IFITM1','CXCR6','KLRC2','ITGA1','CD101','CD69','FOS','GZMK','CD27',
     'HLA-DQA1','GZMB','GZMA','PRF1','TNFRSF9','PRDM1','TOX','TIGIT','LAG3','HAVCR2','PDCD1',
     'ENTPD1','CTLA4','ITGAE','CD38','DUSP4','DUSP1','ATF3','HSPA1A','HSPA1B']

In [ ]:
average_obs,fraction_obs=bc.get_means(tdata, 'leiden')

In [ ]:
mycol=['#6A3A4C','#5A0007','#5A0007','#6A3A4C','#6A3A4C','#5A0007','#004D43','#5A0007','#6B7900','#6A3A4C','#D16100','#D16100','#6B7900']
#mycol=['0','1','3','5','6','7','8','9','10','11','13','15','16']
#mycol=['#6A3A4C','#6A3A4C','#6A3A4C','#6A3A4C','#6A3A4C','#6A3A4C','#004D43','#004D43','#6B7900','#6A3A4C','#6A3A4C','#6A3A4C','#6B7900']

In [ ]:
mycol=pd.Series(mycol)
mycol.index=['0','1','3','5','6','7','8','9','10','11','13','15','16']

In [ ]:
myorder=['16','10','3','7','1','9','8','15','13','6','0','11','5']

In [ ]:
### Plot parameters for publication 
def set_pub():    
    small_size = 10
    medium_size = 12
    large_size = 14

    resolution = 300 #in dpi
    plt.rcParams['font.weight'] = 'normal'
    #plt.rc('font', **{'family':'sans-serif','sans-serif':['Helvetica']})
    plt.rc('axes', titlesize=large_size, titleweight = "bold")               # fontsize of the axes title
    plt.rc('axes', labelsize=medium_size, labelweight = "bold")               # fontsize of the x and y labels
    plt.rc('xtick', labelsize=small_size)               # fontsize of the tick labels
    plt.rc('ytick', labelsize=small_size)               # fontsize of the tick labels
    plt.rc('legend', fontsize=small_size, title_fontsize = medium_size)               # legend fontsize
    plt.rc('figure', titlesize=large_size, titleweight = "bold")              # fontsize of the figure title
    plt.rc('savefig', dpi=resolution)                   # higher res outputs

    plt.rcParams['svg.fonttype'] = 'none'


set_pub()

In [ ]:
sns.set(font_scale=0.6)
a=sns.clustermap(average_obs.loc[myorder,goi],col_cluster=False, row_cluster=False,figsize=(15,4),
                 cmap='viridis',row_colors=list(mycol[myorder]),metric='correlation', 
                 )
#a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.nc.pdf')
#a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.nc.svg')
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.nc.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
a=sns.clustermap(average_obs.loc[myorder,goi],col_cluster=True, row_cluster=False,figsize=(15,4),
                 cmap='viridis',row_colors=list(mycol[myorder]),metric='cosine', standard_scale=1,
                 )
#a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.standard.pdf')
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.standard.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
a=sns.clustermap(average_obs.loc[myorder,goi],col_cluster=False, row_cluster=False,figsize=(15,4),
                 cmap='viridis',row_colors=list(mycol[myorder]),metric='cosine', standard_scale=1,
                 )
#a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.nc.standard.pdf')
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.nc.standard.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
sns.set(font_scale=0.6)
a=sns.clustermap(fraction_obs.loc[:,goi],col_cluster=False, row_cluster=True,figsize=(15,4),
                 cmap='viridis',row_colors=mycol,metric='correlation')
#a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.nc.pdf')
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.nc.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
sc.pl.matrixplot(tdata,var_names=goiclus,vmax=2,groupby='leiden',dendrogram=True)

In [ ]:
sc.pl.matrixplot(tdata,var_names=goi,groupby='leiden',dendrogram=False, 
                 standard_scale='var', categories_order=myorder,
                 save='Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.nc.scanpy_zscore.svg')



In [ ]:
sc.pl.matrixplot(tdata,var_names=['STMN1','BIRC5','MKI67','CCR7','TCF7','SELL','PLAC8','IL7R','LTB','CD160','FCRL6',
                                  'GZMK','GZMB','PRF1','IFNG','XCL1','XCL2','CRTAM','CXCR6','KLRC2','SNAP47','CXCL13',
                                  'PDCD1','TOX','HAVCR2','ATF3','HSPA1A','HSPH1'],vmax=2,groupby='leiden',dendrogram=True)

### Plot and explore other markers 

In [ ]:
#Fairfax et al. https://www.nature.com/articles/s41591-019-0734-6?proof=t
largeclones=['PRF1', 'CCL4', 'GNLY','ITGB1']
#Sade-Feldmann et al. 
sade={}
sade['good']=['PLAC8', 'LTB', 'LY9', 'SELL', 'TCF7',  'CCR7']
sade['bad']=['CCL3', 'CD38', 'HAVCR2', 'ENTPD1', 'WARS']

#Li et al. https://www.sciencedirect.com/science/article/pii/S009286741831568X
#Cyt from li et al. likely correspond to out cluster 7 (and the other NK-Tlike cells that we removed)
#Dys from Li et al. - most of our TIL clusters except for 9, 1, 3
#Nai from Li et al. - 10/2
#Mem from Li et al. - 3/1/9
li={}
li['NaiTcell']=['CCR7','IL7R','TCF7']
li['MemTcell']=[ 'SELL', 'C1orf21', 'KLRB1', 'ARL4C']
li['CD8Cyt']=['GZMH', 'GNLY', 'FGFBP2', 'CX3CR1','KLF2','TBX21', 'PLAC8', 'FGR','SPON2', 'MYBL1','ZNF683','KLRG1']
li['CD8Dys']=['PDCD1','LAG3','TIGIT', 'CXCL13','RBPJ', 'ZBED2', 'ETV1', 'ID3', 'MAF', 'PRDM1','EOMES', 'IFNG', 
              'HAVCR2','PTMS','FAM3C','ICOS','TNFRSF4', 'CCL4L2', 'PRDM1','SPOCK2', 'CCL3', 'TOX', 'ENTPD1','ITGAE']
li['CD8Trans']=['GZMK']
li['CD4Treg']=['FOXP3','IKZF2','IL2RA'] #ENTPD1, ITGAE, KLRG1
li['TExh']=['TNFRSF9', 'CSF1', 'TIGIT']

#Wu et al. https://www.nature.com/articles/s41586-020-2056-8
# EM from Wu et al. corresponds to our exh-like cluster, in particular the NR one; why EM?
# Il7 from Wu et al. is cluster 7, see Cyt from Li et al. above
# RM from Wu et al. could be cluster 8
# Eff Wu et al. could be cluster 0
wucd4=['IL6ST','CRIP1']
wueff=['CX3CR1','GNLY', 'NKG7',  'GZMH', 'KLRD1', 'GZMB', 'PRF1', 
       'IFITM2', 'LITAF','ITGB2','GZMA','GPR56','KLRC2','GZMM','RAP1B']
wuem=['GZMK','CCL4',   'DUSP2', 'CD74','DNAJB1','FOS','CCL3','IFNG']
wurm=['CCL4', 'XCL1',   'XCL2',   'ZNF683']
wuil7=['NCR3','KLRB1','LYAR','IL7R']

litgoi=largeclones+sade['good']+sade['bad']+li['NaiTcell']+li['MemTcell']+li['CD8Cyt']+li['CD8Dys']+li['CD8Trans']+li['TExh']
litgoi=list(set(litgoi+wueff+wuem+wurm+wuil7))

In [ ]:
yost_exh=['Krt86','Layn','Entpd1','Acp5','Galnt2','Tigit','Havcr2','Gzmb','Ahi1','Atp8b4','Itgae',
          'Vcam1','Golim4','Mtss1','Jaml','Sox4','Pde7b','Cxcr6','Csf1','Tnfrsf18','Asb2','Gem','Sla2',
          'Myo7a','Sqle']
yost_act=['Nfkbia','Junb','Jun','Tnf','Ier2','Ifng','Slc2a3','Cd69','Fos','Nr4a2','Ubc','Gadd45b','Nr4a1',
         'Tsc22d3','Hspa1b','Dusp1','Zfp36l1','Ppp1r15a','Actb','Bhlhe40','Fosb','Pim1',
         'Clic1','Hspa1a','Cdkn1a','Nfkbiz','Zc3h12a','Tmsb10','Csrnp1']

In [ ]:
mat_cytotox=['PRF1', 'GZMB', 'GZMA', 'GZMH', 'NKG7', 'GNLY']
mat_nk=['KLRD1', 'FGFBP2', 'FCGR3A', 'S1PR5', 'KLRC1','KLRC3', 'KLRB1',  'KLRC2']
mat_cytotxnk=['GZMB','GNLY', 'NKG7','PRF1', 'KLRD1', 'FGFBP2', 'FCGR3A', 'S100A4','GPR56', 'GZMH']
mat_ifn=['CLIC1', 'CD74', 'PSME2', 'APOBEC3G', 'HLA-DRB1', 'HLA-DRA', 'MT2A', 'HLA-DRB5', 'IFI6', 'CRIP1']
mat_em=['IL7R','GPR183', 'TNFAIP3', 'CD55','TOB1', 'CCR7', 'NFKBIA', 'FOS', 'KLRB1', 'PIM2']
mat_chemokine=['RGS1','CCL4', 'GZMK', 'CCL4L1', 'CCL4L2','CD74', 'ZFP36','DUSP4', 'HLA-DRB1','HLA-DRB5']
mat_stress=['HSPA1B', 'HSPA1A','DNAJB1', 'HSP90AA1', 'HSPH1', 'HSPD1', 'HSPA6', 'HSP90AB1', 'ZFAND2A', 'HSPE1']

In [ ]:
# Signature from https://www.nature.com/articles/nature19330
resource={}
resource['UP']=['TCF7','PDCD1','SLAMF6','CXCR3','CD28','BCL6','PLAGL1']
resource['DN']=['HAVCR2','CD244','ENTPD1']
resourceext={}
resourceext['UP']=['TCF7','PDCD1','SLAMF6','CXCR3','CD28','BCL6','PLAGL1','ICOS','TNFSF14','TNFSF4','IL2','TNF','LAG3', 'CTLA4','BCL6']
resourceext['DN']=['HAVCR2','CD244','ENTPD1','PRDM1','ID2','IL2RB','KLRG1','CCL3','CCL4','CCL5','CSF1']

sigs={}
sigs['resourceCD8Tcell']=resource
sigs['resourceCD8Tcellext']=resourceext

In [ ]:
bc.tl.sig.combined_signature_score(adata, signature_dict=sigs)
sc.tl.score_genes(adata,gene_list=mat_cytotox,score_name='mat_cytotox')
sc.tl.score_genes(adata,gene_list=mat_nk,score_name='mat_nk')
sc.tl.score_genes(adata,gene_list=mat_cytotxnk,score_name='mat_cytotxnk')
sc.tl.score_genes(adata,gene_list=mat_ifn,score_name='mat_ifn')
sc.tl.score_genes(adata,gene_list=mat_em,score_name='mat_em')
sc.tl.score_genes(adata,gene_list=mat_chemokine,score_name='mat_chemokine')
sc.tl.score_genes(adata,gene_list=mat_stress,score_name='mat_stress')

In [ ]:
bc.tl.sig.combined_signature_score(tdata, signature_dict=sigs)
sc.tl.score_genes(tdata,gene_list=mat_cytotox,score_name='mat_cytotox')
sc.tl.score_genes(tdata,gene_list=mat_nk,score_name='mat_nk')
sc.tl.score_genes(tdata,gene_list=mat_cytotxnk,score_name='mat_cytotxnk')
sc.tl.score_genes(tdata,gene_list=mat_ifn,score_name='mat_ifn')
sc.tl.score_genes(tdata,gene_list=mat_em,score_name='mat_em')
sc.tl.score_genes(tdata,gene_list=mat_chemokine,score_name='mat_chemokine')
sc.tl.score_genes(tdata,gene_list=mat_stress,score_name='mat_stress')

In [ ]:
#bc.tl.sig.combined_signature_score(tdata, signature_dict=sigs)
sc.tl.score_genes(adata,gene_list=sigs['resourceCD8Tcell']['UP'],score_name='resourceCD8Tcell_UP')
sc.tl.score_genes(adata,gene_list=sigs['resourceCD8Tcell']['DN'],score_name='resourceCD8Tcell_DOWN')

In [ ]:
#bc.tl.sig.combined_signature_score(tdata, signature_dict=sigs)
sc.tl.score_genes(tdata,gene_list=sigs['resourceCD8Tcell']['UP'],score_name='resourceCD8Tcell_UP')
sc.tl.score_genes(tdata,gene_list=sigs['resourceCD8Tcell']['DN'],score_name='resourceCD8Tcell_DOWN')

In [ ]:
sc.tl.score_genes(tdata,gene_list=yost_act,score_name='Activation_Yost')
sc.tl.score_genes(tdata,gene_list=yost_exh,score_name='Exhaustion_Yost')
#Th17=['Cd163l1','Abi3bp','Il17a','Kcnc1','Rorc', 'Cryba4']
#sc.tl.score_genes(adata,gene_list=Th17,score_name='score_Th17_scanpy')
#https://www.cell.com/cell/pdfExtended/S0092-8674(15)01696-7
#Ifna=['Ifit3','Slfn5','Ifit1','Rsad2','Oas3','Cxcl10','Ifi204','Irf7','Oas2','Rfp4','Stat1','Usp18','Socs1','Gbp4','Ifn7']
#sc.tl.score_genes(adata,gene_list=Ifna,score_name='score_Ifna_scanpy')

In [ ]:
sc.pl.matrixplot(adata, var_names=list(set(sigs['resourceCD8Tcellext']['UP']).union(set(sigs['resourceCD8Tcellext']['DN']))), 
                 groupby='leiden',standard_scale='var',dendrogram=True, save='Heatmap-resource-leiden.svg')

In [ ]:
sc.pl.umap(tdata,color=litgoi, color_map='viridis')

In [ ]:
sc.pl.umap(tdata,color=['IFITM1','GZMA'], color_map='viridis')

In [ ]:
average_obs,fraction_obs=bc.get_means(tdata[tdata.obs['Sample type']=='TIL'], 'leiden')

tdata_mature.uns['celltype3_pub_colors']

mycol=['#6A3A4C','#5A0007','#5A0007','#6A3A4C','#6A3A4C','#5A0007','#004D43','#004D43','#6B7900','#6A3A4C','#D16100','#D16100','#6B7900']
#mycol=['0','1','3','5','6','7','8','9','10','11','13','15','16']
#mycol=['#6A3A4C','#6A3A4C','#6A3A4C','#6A3A4C','#6A3A4C','#6A3A4C','#004D43','#004D43','#6B7900','#6A3A4C','#6A3A4C','#6A3A4C','#6B7900']


In [ ]:

sns.set(font_scale=0.7)
a=sns.clustermap(fraction_obs.loc[:,litgoi+['MKI67','STMN1']],col_cluster=True, row_cluster=True,figsize=(16,4),
                 cmap='viridis',row_colors=mycol,metric='correlation',vmax=1)
a.savefig(figdir+'Heatmap-litgoi-'+velosubset+'.celltype3_pub.tdata_tumorOnly.pdf')
a.savefig(figdir+'Heatmap-litgoi-'+velosubset+'.celltype3_pub.tdata_tumorOnly.svg', format='svg', bbox_inches="tight", dpi=300)

#### Explore differential expression among the clusters 

In [ ]:
### Difference 0,6 vs. 11,5?
#cluster 5 versus 6, cluster 15 R+TF versus both NR, 
#cluster 13 R+TF versus both NR, cluster 9 R+TF versus both NR, 
#cluster 0 R+TF versus both NR, Velo driver genes: 9 to 0, 9 to 11, 0 to 6, 0 to 11. 
    
#subt=tdata[tdata.obs['leiden'].isin(['5','6'])].copy()
subt=tdata[tdata.obs['leiden'].isin(['8'])].copy()
#tmp=subt.obs['leiden'].copy()
#tmp[subt.obs['leiden']=='6']='0'
#tmp[subt.obs['leiden']=='11']='5'
#subt.obs['leidens']=list(tmp)

In [ ]:
set(subt.obs['Respond'])

In [ ]:
DEgenesS=bc.tl.dge.get_de(subt,'Respond',demethod='wilcoxon',topnr=5000, logfc=1,padj=0.05)


In [ ]:
#DEgenes['0']
for i in DEgenesS.keys():
    DEgenesS[i].sort_values('Log2FC',ascending=False).to_csv(figdir+'/DEresponsegenes_c8_'+velosubset+'-'+mysubset+'_'+subcat+'_subdata_leiden_'+i+'.tsv',sep='\t')

In [ ]:
sc.tl.dendrogram(tdata,groupby='leiden')

In [ ]:

### Select only top genes (in order of p-val) for 2 clusters and plot expression per cluster
tops=list(DEgenesS['5'][0:50].sort_values('Log2FC',ascending=False)['Name'])
sc.pl.dotplot(tdata, var_names=tops,groupby='leiden', dendrogram=True)

In [ ]:
tops=list(DEgenesS['0'][0:50].sort_values('Log2FC',ascending=False)['Name'])
sc.pl.dotplot(tdata, var_names=tops,groupby='leiden', dendrogram=True)

In [ ]:
sc.pl.umap(adata,color='leiden')

In [ ]:
#DEgenes=bc.tl.dge.get_de(tdata,'leiden',demethod='wilcoxon',topnr=5000, logfc=1,padj=0.05)
DEgenes=bc.tl.dge.get_de(adata,'leiden',demethod='wilcoxon',topnr=5000, logfc=1,padj=0.05)


In [ ]:
DEgenes=bc.tl.dge.get_de(tdata,'leiden',demethod='wilcoxon',topnr=5000, logfc=1,padj=0.05)




In [ ]:
#DEgenes['0']
for i in DEgenes.keys():
    DEgenes[i].sort_values('Log2FC',ascending=False).to_csv(figdir+'/Topclustergenes_'+velosubset+'-'+mysubset+'_'+subcat+'_tdata_leiden_'+i+'.tsv',sep='\t')

In [ ]:
for i in DEgenes.keys():
    DEgenes[i].sort_values('Log2FC',ascending=False).to_csv(figdir+'/Topclustergenes_'+velosubset+'-'+mysubset+'_'+subcat+'_adata_leiden_'+i+'.tsv',sep='\t')

In [ ]:
### Select only top genes (in order of p-val) for 2 clusters and plot expression per cluster
tops=list(DEgenes['14'][0:50].sort_values('Log2FC',ascending=False)['Name'])
sc.pl.dotplot(tdata, var_names=tops,groupby='leiden', dendrogram=True)

In [ ]:
### Select only top genes (in order of p-val) for 2 clusters and plot expression per cluster
tops=list(DEgenes['13'][0:50].sort_values('Log2FC',ascending=False)['Name'])
sc.pl.dotplot(tdata, var_names=tops,groupby='leiden', dendrogram=True)

In [ ]:
### Select only top genes (in order of p-val) for 2 clusters and plot expression per cluster
tops=list(DEgenes['3'][0:50].sort_values('Log2FC',ascending=False)['Name'])
sc.pl.dotplot(tdata, var_names=tops,groupby='leiden', dendrogram=True)

In [ ]:
sc.set_figure_params()

In [ ]:
cr.pl.initial_states(tdata, discrete=True, save='scvelo_Initial_states_CD8Tcell.PAGA.svg')


In [ ]:
cr.pl.terminal_states(tdata, discrete=True, save='scvelo_Terminal_states_CD8Tcell.PAGA.svg')


In [ ]:
cr.tl.lineages(tdata)
cr.pl.lineages(tdata, same_plot=False)



In [ ]:
#scv.tl.recover_latent_time(tdata, root_key='initial_states_probs', end_key='terminal_states_probs')


In [ ]:
scv.tl.recover_latent_time(tdata, root_key='initial_states_probs', end_key='terminal_states_probs')



In [ ]:
scv.tl.paga(tdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)



cr.pl.cluster_fates(tdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA', 
                   save='DirectedGraphCD8Tcell.PAGA.svg')


In [ ]:
cr.pl.lineages(tdata, same_plot=False)

In [ ]:
sc.pl.umap(tdata,color=['latent_time'])

#### Stratify analysis per response class 

In [ ]:
rdata=tdata[tdata.obs['RCat'].isin(['R'])].copy()
tfdata=tdata[tdata.obs['RCat'].isin(['TF'])].copy()

nrdata=tdata[tdata.obs['RCat'].isin(['NR_adj'])].copy()
nrnadjdata=tdata[tdata.obs['RCat'].isin(['NR_nadj'])].copy()

In [ ]:
cr.pl.initial_states(rdata, discrete=True)

In [ ]:
len(rdata)

In [ ]:
len(tfdata)

In [ ]:
len(nrdata)

In [ ]:
len(nrnadjdata)

In [ ]:
nrbdata=tdata[tdata.obs['RCat'].isin(['NR_adj','NR_nadj'])].copy()
rbdata=tdata[tdata.obs['RCat'].isin(['R','TF'])].copy()


In [ ]:
scv.tl.paga(rbdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)



cr.pl.cluster_fates(rbdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA', 
                   save='DirectedGraphCD8Tcell.PAGA.rbdata.svg')


In [ ]:
cnames=['0','1','3','5','6','7','8','9','10','11','13','15','16']
tconf=scv.get_df(rbdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_rbdata.tsv',sep='\t')

In [ ]:
scv.tl.paga(nrbdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)



cr.pl.cluster_fates(nrbdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA',
                    save='DirectedGraphCD8Tcell.PAGA.nrbdata.svg')


In [ ]:
tconf=scv.get_df(nrbdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_nrbdata.tsv',sep='\t')

In [ ]:
tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


#### Redo the same analysis but only with a subset of clusters - remove naive/memory T cells

In [ ]:
nrbdata_mature=nrbdata[nrbdata.obs['leiden'].isin(['9','8','11','6','5','13','15','0'])].copy()
rbdata_mature=rbdata[rbdata.obs['leiden'].isin(['9','8','11','6','5','13','15','0'])].copy()

In [ ]:
tdata_mature=tdata[tdata.obs['leiden'].isin(['9','8','11','6','5','13','15','0'])].copy()

In [ ]:
scv.tl.paga(tdata_mature, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)



cr.pl.cluster_fates(tdata_mature, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA',
                    save='DirectedGraphCD8Tcell.PAGA.tdata_mature.svg')


In [ ]:
cr.tl.lineage_drivers(tdata_mature)

lin_drivers_sub=tdata_mature.raw.var['to 5']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]

In [ ]:
scv.tl.paga(nrbdata_mature, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)



cr.pl.cluster_fates(nrbdata_mature, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA',
                   save='DirectedGraphCD8Tcell.PAGA.nrbdata_mature.svg')


In [ ]:
cnames=['0','5','6','8','9','11','13','15']
tconf=scv.get_df(nrbdata_mature, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_nrbdata_mature.tsv',sep='\t')

In [ ]:
tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
cr.tl.lineage_drivers(nrbdata_mature)

lin_drivers_sub=nrbdata_mature.raw.var['to 5']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]

In [ ]:
lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=True)[0:15]

In [ ]:
scv.tl.paga(rbdata_mature, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)



cr.pl.cluster_fates(rbdata_mature, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA',
                   save='DirectedGraphCD8Tcell.PAGA.rbdata_mature.svg')


In [ ]:
cnames=['0','5','6','8','9','11','13','15']
tconf=scv.get_df(rbdata_mature, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_rbdata_mature.tsv',sep='\t')

In [ ]:
tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
cr.tl.lineage_drivers(rbdata_mature)

lin_drivers_sub=rbdata_mature.raw.var['to 5']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]

In [ ]:
len(nrbdata)

#### Get the transition confidence 

In [ ]:
scv.tl.paga(rdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')

cr.pl.cluster_fates(rdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA')


In [ ]:
cnames=['0','1','3','5','6','7','8','9','10','11','13','15','16']
tconf=scv.get_df(rdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_rdata.tsv',sep='\t')
tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
scv.tl.paga(tfdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')

cr.pl.cluster_fates(tfdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA')


In [ ]:
cnames=['0','1','3','5','6','7','8','9','10','11','13','15','16']
tconf=scv.get_df(tfdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_tfdata.tsv',sep='\t')
tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
scv.tl.paga(nrdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')

cr.pl.cluster_fates(nrdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA')


In [ ]:
cnames=['0','1','3','5','6','7','8','9','10','11','13','15','16']
tconf=scv.get_df(nrdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_nrdata.tsv',sep='\t')
tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
scv.tl.paga(nrnadjdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')

cr.pl.cluster_fates(nrnadjdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA')


In [ ]:
cnames=['0','1','3','5','6','7','8','9','10','11','13','15','16']
tconf=scv.get_df(nrnadjdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_nrnadjdata.tsv',sep='\t')
tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


#### Expore values per individual patient as well 

In [ ]:
cnames=['0','1','3','5','6','7','8','9','10','11','13','15','16']

In [ ]:
Resp=[tdata[tdata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0] for x in list(set(tdata.obs['PatientID']))]

In [ ]:
pattrans={}
for i in list(set(tdata.obs['PatientID'])):
    sub=tdata[tdata.obs['PatientID']==i].copy()
    scv.tl.paga(sub, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')
    tmp=scv.get_df(sub, 'paga/transitions_confidence', precision=2).T
    sc.pl.umap(sub,color='leiden')
    tmp.index=list(sub.obs['leiden'].cat.categories)
    tmp.columns=list(sub.obs['leiden'].cat.categories)
    pattrans[i]=pd.DataFrame(0,index=cnames,columns=cnames).add(tmp, fill_value=0)

In [ ]:
### Version only with Mature data
pattrans={}
for i in list(set(tdata_mature.obs['PatientID'])):
    sub=tdata_mature[tdata_mature.obs['PatientID']==i].copy()
    scv.tl.paga(sub, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')
    tmp=scv.get_df(sub, 'paga/transitions_confidence', precision=2).T
    sc.pl.umap(sub,color='leiden')
    tmp.index=list(sub.obs['leiden'].cat.categories)
    tmp.columns=list(sub.obs['leiden'].cat.categories)
    pattrans[i]=pd.DataFrame(0,index=cnames,columns=cnames).add(tmp, fill_value=0)

In [ ]:
pattrans.keys()

In [ ]:
for i in pattrans.keys():
    print(i)
    toplot=pattrans[i].loc[pattrans[i].mean()>0,:].copy()
    toplot=toplot.loc[:,toplot.mean()>0].copy()
    plt.figure()
    a=sns.heatmap(toplot, annot=True, linewidths=.5, annot_kws={"size":10})
    fig = a.get_figure()
    fig.savefig(figdir+'Transitions-perPatient-Heatmap-'+velosubset+'.PatientNr-'+i+'.PBMCandTIL.tdata.pdf')

In [ ]:
Rs=pd.DataFrame(Resp).copy()
Rs.index=list(set(tdata.obs['PatientID']))

In [ ]:
toC8={}
for k in pattrans.keys():
    toC8[k]=pattrans[k].loc[:,'8']
toC8=pd.melt(pd.DataFrame(toC8).reset_index(), id_vars='index')
toC8.columns=['leiden','Patient','Transition']
toC8['RCat']=list(Rs.loc[list(toC8.Patient),:].iloc[:,0])

In [ ]:
subplot=toC6.copy()
cchoice=['13']
outtxt='13to6'
a=sns.boxplot(data=subplot.loc[subplot.leiden.isin(cchoice),:],x='RCat',y='Transition',
            order=['R','TF','NR_nadj','NR_adj'],palette=color_dict)
a=sns.swarmplot(data=subplot.loc[subplot.leiden.isin(cchoice),:],x='RCat',y='Transition',
              color=".25",order=['R','TF','NR_nadj','NR_adj'])
fig = a.get_figure()
#fig.savefig(figdir+'Transitions-perPatient-'+velosubset+'.8and9to11.PBMCandTIL.tdata.pdf')
fig.savefig(figdir+'Transitions-perPatient-'+velosubset+'.'+outtxt+'.PBMCandTIL.tdata.pdf')


In [ ]:
mtoC11=toC11.loc[toC11.leiden.isin(['8','9']),:].groupby('RCat')['Transition'].mean()
mtoC6=toC6.loc[toC6.leiden.isin(['0']),:].groupby('RCat')['Transition'].mean()
mtoC0=toC0.loc[toC0.leiden.isin(['8','9']),:].groupby('RCat')['Transition'].mean()
mtoC5=toC5.loc[toC5.leiden.isin(['11','9']),:].groupby('RCat')['Transition'].mean()

In [ ]:
trans=pd.DataFrame([mtoC11,mtoC5,mtoC6,mtoC0])
trans.index=['toC11','toC5','toC6','toC0']

In [ ]:
a=sns.heatmap(trans, annot=True, linewidths=.5, annot_kws={"size":10})
fig = a.get_figure()
#fig.savefig(figdir+'Transitions-MeanperPatient-'+velosubset+'.4paths.PBMCandTIL.tdata.pdf')
fig.savefig(figdir+'Transitions-MeanperPatient-'+velosubset+'.4paths.TILonly.tdata.pdf')


### Check differences between cell nrs. per Response Category

In [ ]:
from scipy import stats
import itertools


def getPs(cellFreqs,myconditions,name1):
    totest=list(itertools.combinations(myconditions, 2))
    pwilc={}
    pt={}
    for pairs in totest:
        pwilc[pairs[0]+'-'+pairs[1]]=stats.mannwhitneyu(list(cellFreqs.loc[cellFreqs[name1]==pairs[0],:].iloc[:,1]), 
                           list(cellFreqs.loc[cellFreqs[name1]==pairs[1],:].iloc[:,1]))[1]
        pt[pairs[0]+'-'+pairs[1]]=stats.ttest_ind(list(cellFreqs.loc[cellFreqs[name1]==pairs[0],:].iloc[:,1]), 
                           list(cellFreqs.loc[cellFreqs[name1]==pairs[1],:].iloc[:,1]))[1]

    myps=pd.DataFrame([pwilc,pt]).transpose()
    myps.columns=['MannWhitney','T-test']
    return(myps)


In [ ]:
#sc.pl.umap(adata,color='leiden')
tildata=adata[adata.obs['Sample type']=='TIL'].copy()

In [ ]:
tildata.obs['gleiden']=list(tildata.obs['leiden'])

In [ ]:
newc=tildata.obs['gleiden'].copy()
newc=newc.replace('4', '5').copy()
newc=newc.replace('2', '5').copy()
newc=newc.replace('1', '3').copy()
newc=newc.replace('14', '13').copy()
newc=newc.replace('12', '13').copy()
newc=newc.replace('15', '13').copy()

In [ ]:
tildata.obs['gleiden']=list(newc)

In [ ]:
sc.pl.umap(tildata,color='gleiden', legend_loc='on data')

In [ ]:
sc.pl.umap(tildata,color='leiden', legend_loc='on data')

In [ ]:
propdir=figdir+'/frequencies/'

#mysubs=['celltype4_pub','celltype3_pub','celltype2_pub', 'celltype1']
#mysubs=['leiden','gleiden']
mysubs=['gleiden']
#mysubs=['celltype3_pub']
for what in mysubs:    
    df1=bc.tl.count_occurrence_subset_conditions(tildata, subset_variable = 'PatientID', count_variable = what, condition_identifier = 'RCat',  return_percentage = True)
    df1.to_csv(propdir+velosubset+'TILtdata_celltypeFreq_'+what+'.tsv')
    df2=bc.tl.count_occurrence_subset_conditions(tildata, subset_variable = 'PatientID', count_variable = what, condition_identifier = 'RCat',  return_percentage = False)
    df2.to_csv(propdir+velosubset+'TILtdata_celltypeNrs_'+what+'.tsv')
    


In [ ]:
what='gleiden'
tilfreq=pd.read_csv(propdir+velosubset+'TILtdata_celltypeFreq_'+what+'.tsv')

tilfreq.index=tilfreq.iloc[:,0]


rs=[x.split(' ')[2] for x in list(tilfreq.drop(columns=['Unnamed: 0']).columns)]
patid=[x.split(' ')[1] for x in list(tilfreq.drop(columns=['Unnamed: 0']).columns)]

toplot=tilfreq.drop(columns=['Unnamed: 0']).transpose()
toplot['RCat']=rs



fig, axes = plt.subplots(3, 6,figsize=(16,12), gridspec_kw={'wspace': 0.5, 'left': 0.25})
plt.subplots_adjust(left=0.2, right=0.98, top=0.86, bottom=0.1)

axes = axes.flatten()
pvalsn={}
pvals={}
i=0
for mycell in list(tilfreq.index):
    ax=sns.boxplot(y=mycell,x='RCat',data=toplot,ax=axes[i],
                   palette=color_dict,orient='v',order=['R','TF','NR_nadj','NR_adj'])
    ax=sns.swarmplot(y=mycell,x='RCat',data=toplot,color='black',ax=axes[i]
                    ,order=['R','TF','NR_nadj','NR_adj'])
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90,fontsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.set_ylabel(ylabel='Perc '+str(mycell),fontsize=10)
    ax.set_xlabel(xlabel='Response ',fontsize=10)
    prov=toplot.loc[:,['RCat',mycell]].groupby('RCat')[mycell].mean()
    if len(prov[prov==0])<2:
        pvalsn[mycell]=getPs(toplot.loc[:,['RCat',mycell]],['R','TF','NR_nadj','NR_adj'],'RCat').loc['R-NR_nadj',:]
        pvals[mycell]=getPs(toplot.loc[:,['RCat',mycell]],['R','TF','NR_nadj','NR_adj'],'RCat').loc['TF-NR_adj',:]

    i=i+1


In [ ]:
fig.savefig(figdir+'Celltypefreq-Response-mature-'+what+velosubset+'.TILonly.pdf')

In [ ]:
pd.DataFrame.from_dict(pvalsn, orient='index').sort_values(by='MannWhitney')

In [ ]:
pd.DataFrame.from_dict(pvals, orient='index').sort_values(by='MannWhitney')

In [ ]:
pd.DataFrame.from_dict(pvals, orient='index').to_csv(figdir+'Pval-CellTypeFreq-TIL-'+velosubset+what+'-per-Response-adj.tsv',sep='\t')

pd.DataFrame.from_dict(pvalsn, orient='index').to_csv(figdir+'Pval-CellTypeFreq-TIL-'+velosubset+what+'-per-Response-nadj.tsv',sep='\t')


In [ ]:
DEgenesS=bc.tl.dge.get_de(tildata[tildata.obs['leiden'].isin(['0','5'])],'gleiden',demethod='wilcoxon',topnr=5000, logfc=1,padj=0.05)


In [ ]:
### Select only top genes (in order of p-val) for 2 clusters and plot expression per cluster
tops=list(DEgenesS['5'][0:50].sort_values('Log2FC',ascending=False)['Name'])
sc.pl.dotplot(tdata, var_names=tops,groupby='leiden', dendrogram=True)


In [ ]:
tops=list(DEgenesS['0'][0:50].sort_values('Log2FC',ascending=False)['Name'])
sc.pl.dotplot(tdata, var_names=tops,groupby='leiden', dendrogram=True)


In [ ]:
nr33=tdata[tdata.obs['PatientID']=='33'].copy()
scv.tl.paga(nr33, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')

cr.pl.cluster_fates(nr33, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA')


### Lineage drivers 

#### Obtain velo driver genes for specific subsets of the data

In [ ]:
subp=rbdata[rbdata.obs['leiden'].isin(['8','11','5'])].copy()
#subp=nrbdata[nrbdata.obs['leiden'].isin(['8','0','6'])].copy()
#subp=nrbdata[nrbdata.obs['leiden'].isin(['15','13','0'])].copy()
#subp=rbdata[rbdata.obs['leiden'].isin(['9','0','6'])].copy()
#subp=nrbdata[nrbdata.obs['leiden'].isin(['9','11','5'])].copy()
#subp=rbdata[rbdata.obs['leiden'].isin(['8','13','0','6'])].copy()
#subp=nrbdata[nrbdata.obs['leiden'].isin(['8','13','0','11','5'])].copy()
#subp=tdata[tdata.obs['leiden'].isin(['0','6'])].copy()
#subp=tdata[tdata.obs['leiden'].isin(['15','13','0'])].copy()

scv.tl.paga(subp, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


In [ ]:
sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)


In [ ]:

cr.pl.cluster_fates(subp, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA')


In [ ]:
cr.tl.lineage_drivers(subp)

In [ ]:
lin_drivers_sub=subp.raw.var['to 5']

In [ ]:
lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:20]

In [ ]:
lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=True)[0:20]

In [ ]:
#lin_drivers_sub[np.abs(lin_drivers_sub)>0.1]

In [ ]:
#lin_drivers_sub[np.abs(lin_drivers_sub)>0.1].sort_values(ascending=False).to_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_tdata_leiden_0to6.tsv',sep='\t')
#lin_drivers_sub[np.abs(lin_drivers_sub)>0.1].sort_values(ascending=False).to_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_nrbdata_leiden_8to0and6.tsv',sep='\t')
lin_drivers_sub[np.abs(lin_drivers_sub)>0.1].sort_values(ascending=False).to_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_rbdata_leiden_8to11and5.tsv',sep='\t')

In [ ]:
#lindnr=pd.read_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_nrbdata_leiden_9to5inNRs.tsv',sep='\t')
#lindnr=pd.read_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_15to11inNRs.tsv',sep='\t')
#lindr=pd.read_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_15to11inRTFs.tsv',sep='\t')
lindr=pd.read_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_rdata_leiden_0to6inRTFs.tsv',sep='\t')


In [ ]:
cdata=subp[subp.obs['Sample type']=='TIL'].copy()
#=bc.tl.dge.get_de(subdataa,'RCat',demethod='wilcoxon',topnr=5000, logfc=np.log(1.5),padj=0.1)
### try on individual tissues separately
b1DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['LN'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
b2DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['Brain'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
b3DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['Lung','Sinon','Subc'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)

allDE1=bc.tl.dge.get_de(cdata[cdata.obs['Response3'].isin(['R','NR'])],'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
allDE2=bc.tl.dge.get_de(cdata[cdata.obs['Response3'].isin(['TF','PD'])],'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
allDE=bc.tl.dge.get_de(cdata,'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)

#=bc.tl.dge.get_de(subdataa,'RCat',demethod='wilcoxon',topnr=5000, logfc=np.log(1.5),padj=0.1)
### try on individual tissues separately
b1DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['LN'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
b2DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['Brain'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
b3DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['Lung','Sinon','Subc'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)

allDE1=bc.tl.dge.get_de(cdata[cdata.obs['Response3'].isin(['R','NR'])],'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
allDE2=bc.tl.dge.get_de(cdata[cdata.obs['Response3'].isin(['TF','PD'])],'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
allDE=bc.tl.dge.get_de(cdata,'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)


In [ ]:
### Intersection of all combinations
strDE=set(b1DE['PD']['Name']).intersection(set(allDE['PD']['Name'])).intersection((set(b2DE['PD']['Name'])).intersection(set(b3DE['PD']['Name']))).intersection((set(allDE1['PD']['Name'])).intersection(set(allDE2['PD']['Name'])))
strDE2=set(b1DE['R']['Name']).intersection(set(allDE['R']['Name'])).intersection((set(b2DE['R']['Name'])).intersection(set(b3DE['R']['Name']))).intersection((set(allDE1['R']['Name'])).intersection(set(allDE2['R']['Name'])))

strDE

In [ ]:
strDE2

In [ ]:
strDE.intersection(set(lindnr.iloc[:,0]))

In [ ]:
strDE2.intersection(set(lindr.iloc[:,0]))

In [ ]:
### Complete comparison + LN only + Brain | Others + RvsNR | PDvsTF
strDE=set(b1DE['PD']['Name']).intersection(set(allDE['PD']['Name'])).intersection((set(b2DE['PD']['Name'])).union(set(b3DE['PD']['Name']))).intersection((set(allDE1['PD']['Name'])).union(set(allDE2['PD']['Name'])))
strDE2=set(b1DE['R']['Name']).intersection(set(allDE['R']['Name'])).intersection((set(b2DE['R']['Name'])).union(set(b3DE['R']['Name']))).intersection((set(allDE1['R']['Name'])).union(set(allDE2['R']['Name'])))

#strDE


In [ ]:
len(strDE.intersection(set(lindnr.iloc[:,0])))

In [ ]:
strDE2

In [ ]:
strDE2.intersection(set(lindr.iloc[:,0]))

In [ ]:
set(lindr.iloc[:,0])

In [ ]:
strDE.intersection(set(lindr.iloc[:,0]))

In [ ]:
set(lindr.iloc[:,0])

In [ ]:
strDE.intersection(set(lindr.iloc[:,0]))


In [ ]:
strDE2
set()


In [ ]:
sc.set_figure_params()

In [ ]:
#scv.pl.scatter(tdata, ['CCL3L3'], color=['leiden', 'velocity'])

In [ ]:
scv.pl.velocity(tdata, ['CXCR6', 'SNAP47','TIGIT','FAM3C','APOBEC3G','FCRL6'], ncols=2, add_outline=True)

In [ ]:
scv.pl.velocity(tdata_mature, ['ATF3','CCR7','SELL','CXCR6','STMN1','SNAP47'], ncols=2, add_outline=True,color='leiden')

In [ ]:
#rlin9to06=set(tdata.var.loc[tdata.var['velocity_genes']==True,:].index).intersection(set(lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].index))
#rlin8to06=set(tdata.var.loc[tdata.var['velocity_genes']==True,:].index).intersection(set(lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].index))
#nrlin9to115=set(tdata.var.loc[tdata.var['velocity_genes']==True,:].index).intersection(set(lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].index))
#nrlin13to11=set(tdata.var.loc[tdata.var['velocity_genes']==True,:].index).intersection(set(lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].index))

In [ ]:
lindnr=pd.read_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_nrbdata_leiden_9to5inNRs.tsv',sep='\t')
lindr=pd.read_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_rdata_leiden_0to6inRTFs.tsv',sep='\t')


In [ ]:
ddata=subp[subp.obs['Sample type'].isin(['TIL'])].copy()


ddata.obs['R_clus']=ddata.obs['RCat'].astype(str)+'-'+[x.replace("-", "_") for x in ddata.obs['leiden'].astype(str)]
ddata.obs['Pat_clus']=ddata.obs['PatientID'].astype(str)+'-'+[x.replace("-", "_") for x in ddata.obs['leiden'].astype(str)]
dfPat = ddata.obs.groupby('Pat_clus')['velocity_length', 'velocity_confidence','latent_time'].mean().T

#dfPat.to_csv(figdir+'Velocity-per_patient'+velosubset+'-'+mysubset+'-'+subcat+'.tsv',sep='\t')

Resp=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0] for x in list(dfPat.columns)]
PatientID=[x.split('-')[0]  for x in list(dfPat.columns)]
ClusterID=[x.split('-')[1]  for x in list(dfPat.columns)]

mydf=pd.DataFrame([list(dfPat.loc['velocity_length',:]),Resp,PatientID,ClusterID]).transpose()
mydf.index=dfPat.columns
mydf.columns=['velocity_length','Response','PatientID','Cluster']

mydfl=pd.DataFrame([list(dfPat.loc['latent_time',:]),Resp,PatientID,ClusterID]).transpose()
mydfl.index=dfPat.columns
mydfl.columns=['latent_time','Response','PatientID','Cluster']

mydf.groupby('PatientID', as_index=False)['velocity_length']
mydfl.groupby('PatientID', as_index=False)['latent_time']

mydf['velocity_length']=mydf['velocity_length'].astype('float')
mydfl['latent_time']=mydfl['latent_time'].astype('float')

color_dict = {'R': 'coral', 'TF': 'firebrick', 'NR_nadj': 'lightskyblue','NR_adj': 'royalblue'}

In [ ]:
#corder=['8','6','1','21']

#velMean=mydf.groupby('PatientID')['velocity_length'].mean()
velMean=mydfl.groupby('PatientID')['latent_time'].mean()
patid=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0]  for x in list(velMean.index)]
totest=pd.DataFrame([list(velMean),patid]).transpose()
totest.columns=['Val','Cat']

ss.mannwhitneyu(list(totest.loc[totest['Cat']=='R',:]['Val']),list(totest.loc[totest['Cat']=='NR_nadj',:]['Val']))


In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='TF',:]['Val']),list(totest.loc[totest['Cat']=='NR_adj',:]['Val']))



In [ ]:
#totest

In [ ]:
#set(mydf['Response'])
### Velocity length
totestk=totest.copy()

fig=sns.boxplot(y='Val',x='Cat',data=totestk,palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
fig=sns.swarmplot(x='Cat',y='Val',data=totestk,color='black',order=['R','TF','NR_nadj','NR_adj'])
#fig.figure.savefig(figdir+'Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap.pdf') 


In [ ]:


totest=list(itertools.combinations(['R','TF','NR_nadj','NR_adj'], 2))
pwilc={}
pt={}
for pairs in totest:
    pwilc[pairs[0]+'-'+pairs[1]]=stats.mannwhitneyu(list(totestk.loc[totestk['Cat']==pairs[0],:]['Val']), 
                           list(totestk.loc[totestk['Cat']==pairs[1],:]['Val']))[1]
    pt[pairs[0]+'-'+pairs[1]]=stats.ttest_ind(list(totestk.loc[totestk['Cat']==pairs[0],:]['Val']), 
                           list(totestk.loc[totestk['Cat']==pairs[1],:]['Val']))[1]

myps=pd.DataFrame([pwilc,pt]).transpose()
myps.columns=['MannWhitney','T-test']

#myps.to_csv(figdir+'Pval-Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap.pdf',sep='\t')

myps

In [ ]:
cdata=ddata.copy()
#=bc.tl.dge.get_de(subdataa,'RCat',demethod='wilcoxon',topnr=5000, logfc=np.log(1.5),padj=0.1)
### try on individual tissues separately
b1DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['LN'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
b2DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['Brain'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
b3DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['Lung','Sinon','Subc'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)

allDE1=bc.tl.dge.get_de(cdata[cdata.obs['Response3'].isin(['R','NR'])],'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
allDE2=bc.tl.dge.get_de(cdata[cdata.obs['Response3'].isin(['TF','PD'])],'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
allDE=bc.tl.dge.get_de(cdata,'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)

#=bc.tl.dge.get_de(subdataa,'RCat',demethod='wilcoxon',topnr=5000, logfc=np.log(1.5),padj=0.1)
### try on individual tissues separately
b1DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['LN'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
b2DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['Brain'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
b3DE=bc.tl.dge.get_de(cdata[cdata.obs['Lesion'].isin(['Lung','Sinon','Subc'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)

allDE1=bc.tl.dge.get_de(cdata[cdata.obs['Response3'].isin(['R','NR'])],'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
allDE2=bc.tl.dge.get_de(cdata[cdata.obs['Response3'].isin(['TF','PD'])],'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
allDE=bc.tl.dge.get_de(cdata,'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)


In [ ]:
lindnr=pd.read_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_3to9inNRs.tsv',sep='\t')
lindr=pd.read_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_3to9inRTF.tsv',sep='\t')

#Lingenes_CD8Tcell-All_PBMCandTIL_leiden_3to9inNRs.tsv

In [ ]:
### Intersection of all combinations
strDE=set(b1DE['PD']['Name']).intersection(set(allDE['PD']['Name'])).intersection((set(b2DE['PD']['Name'])).intersection(set(b3DE['PD']['Name']))).intersection((set(allDE1['PD']['Name'])).intersection(set(allDE2['PD']['Name'])))
strDE2=set(b1DE['R']['Name']).intersection(set(allDE['R']['Name'])).intersection((set(b2DE['R']['Name'])).intersection(set(b3DE['R']['Name']))).intersection((set(allDE1['R']['Name'])).intersection(set(allDE2['R']['Name'])))



In [ ]:
strDE

In [ ]:
strDE.intersection(set(lindnr.iloc[:,0]))

In [ ]:
strDE.intersection(set(lindr.iloc[:,0]))

In [ ]:
#set(lindr.iloc[:,0])

In [ ]:
strDE2

In [ ]:
### Complete comparison + LN only + Brain | Others + RvsNR | PDvsTF
strDE=set(b1DE['PD']['Name']).intersection(set(allDE['PD']['Name'])).intersection((set(b2DE['PD']['Name'])).union(set(b3DE['PD']['Name']))).intersection((set(allDE1['PD']['Name'])).union(set(allDE2['PD']['Name'])))
strDE2=set(b1DE['R']['Name']).intersection(set(allDE['R']['Name'])).intersection((set(b2DE['R']['Name'])).union(set(b3DE['R']['Name']))).intersection((set(allDE1['R']['Name'])).union(set(allDE2['R']['Name'])))

#strDE


In [ ]:
strDE.intersection(set(lindnr.iloc[:,0]))

In [ ]:
strDE2.intersection(set(lindr.iloc[:,0]))

In [ ]:
#subp=nrbdata[nrbdata.obs['leiden'].isin(['3','1','9'])].copy()
subp=rbdata[rbdata.obs['leiden'].isin(['15','13','0','11','6'])].copy()


In [ ]:
scv.tl.paga(subp, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


In [ ]:
cr.pl.cluster_fates(subp, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA')


In [ ]:
cr.tl.lineage_drivers(subp)
lin_drivers_sub=subp.raw.var['to 5']

In [ ]:
lin_drivers_sub[np.abs(lin_drivers_sub)>0.1].sort_values(ascending=False)


In [ ]:
lin_drivers_sub[np.abs(lin_drivers_sub)>0.1].sort_values(ascending=True)[0:20]

In [ ]:
lin_drivers_sub[np.abs(lin_drivers_sub)>0.1].sort_values(ascending=False)[0:20]

In [ ]:
lin_drivers_sub[np.abs(lin_drivers_sub)>0.1].sort_values(ascending=False).to_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden_15to11inRTFs.tsv',sep='\t')

In [ ]:
sc.pl.umap(subp,color=['LTB','EEF1A1','GLTSCR2','HSPA1A','DNAJA1','DNAJB1','LMNA','KLF6'], color_map='viridis')

In [ ]:
sc.pl.umap(subp,color=['IL7R','LTB','EEF1A1','GLTSCR2','NKG7','CCL4','GZMH','GZMA'], color_map='viridis')

In [ ]:
#subp=nrbdata[nrbdata.obs['leiden'].isin(['3','1','9'])].copy()
subp=nrbdata[nrbdata.obs['Sample type'].isin(['PBMC'])].copy()


scv.tl.paga(subp, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


cr.pl.cluster_fates(subp, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA')




In [ ]:
cr.tl.lineage_drivers(subp)
lin_drivers_sub=subp.raw.var['to 5']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.1].sort_values(ascending=False)


## Analysis per response subtype

In [ ]:
scv.tl.paga(rdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')
scv.tl.paga(tfdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


In [ ]:
scv.tl.paga(nrdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')
scv.tl.paga(nrnadjdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


In [ ]:
scv.tl.paga(nrbdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


In [ ]:
#sc.set_figure_params(10)
#import matplotlib.pyplot as plt
#plt.rcParams["figure.figsize"] = (12,10)

cr.pl.cluster_fates(nrdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4,  title='directed PAGA', 
                    save='-subsettrajectorymap-'+velosubset+'.nrdata.PAGA.pdf')


In [ ]:
cr.pl.cluster_fates(nrnadjdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4,  title='directed PAGA', 
                    save='-subsettrajectorymap-'+velosubset+'.nrnadjdata.PAGA.pdf')


In [ ]:
cr.pl.cluster_fates(rdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4,  title='directed PAGA', 
                    save='-subsettrajectorymap-'+velosubset+'.rdata.PAGA.pdf')


In [ ]:
cr.pl.cluster_fates(tfdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4,  title='directed PAGA',
                    save='-subsettrajectorymap-'+velosubset+'.tfdata.PAGA.pdf')


In [ ]:
sc.pl.violin(tdata, keys='latent_time',groupby='celltype3_pub',
             order=['naive CD8-positive T cell',
                    'effector memory CD8-positive T cell',
                    'proliferating CD8-positive T cell',
                    'cytokine secreting effector CD8-positive T cell',
 'exhausted-like CD8-positive T cell'],rotation=90, save='-latent_time_per_celltype.tdata.svg')

In [ ]:
color_dict

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 7,4
sc.pl.violin(tdata[tdata.obs['Sample type']=='TIL'], keys='latent_time',groupby='PatientID',
             order=['63','72','40','86','33','91','29','87',
                    '13','34','67','68','69','83','79','82','11','43','64','77'],
            palette=['coral','coral','coral','lightskyblue','lightskyblue','lightskyblue','lightskyblue','lightskyblue',
                  'firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick',
                  'royalblue','royalblue','royalblue'],save='-latent_time_per_patient.tdata_TIL.svg')

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 7,4
sc.pl.violin(tdata[tdata.obs['leiden'].isin(['8','9'])], keys='latent_time',groupby='PatientID',
             order=['63','72','40','86','33','91','29','87',
                    '13','34','67','68','69','83','79','82','11','43','64','77'],
            palette=['coral','coral','coral','lightskyblue','lightskyblue','lightskyblue','lightskyblue','lightskyblue',
                  'firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick',
                  'royalblue','royalblue','royalblue'],save='-latent_time_per_patient.tdata_C8-9.svg')

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 7,4
sc.pl.violin(tdata[tdata.obs['leiden'].isin(['13','15'])], keys='latent_time',groupby='PatientID',
             order=['63','72','40','86','33','91','29','87',
                    '13','34','67','68','69','83','79','82','11','43','64','77'],
            palette=['coral','coral','coral','lightskyblue','lightskyblue','lightskyblue','lightskyblue','lightskyblue',
                  'firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick',
                  'royalblue','royalblue','royalblue'])

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 7,4
sc.pl.violin(tdata[tdata.obs['leiden'].isin(['0','6','11','5'])], keys='latent_time',groupby='PatientID',
             order=['63','72','40','86','33','91','29','87',
                    '13','34','67','68','69','83','79','82','11','43','64','77'],
            palette=['coral','coral','coral','lightskyblue','lightskyblue','lightskyblue','lightskyblue','lightskyblue',
                  'firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick',
                  'royalblue','royalblue','royalblue'])

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 7,4
sc.pl.violin(tdata[tdata.obs['leiden'].isin(['10','1','3'])], keys='latent_time',groupby='PatientID',
             order=['63','72','40','86','33','91','29','87',
                    '13','34','67','68','69','83','79','82','11','43','64','77'],
            palette=['coral','coral','coral','lightskyblue','lightskyblue','lightskyblue','lightskyblue','lightskyblue',
                  'firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick',
                  'royalblue','royalblue','royalblue'])

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 7,4
sc.pl.violin(tdata[tdata.obs['leiden'].isin(['8','9','13','15'])], keys='velocity_length',groupby='PatientID',
             order=['63','72','40','86','33','91','29','87',
                    '13','34','67','68','69','83','79','82','11','43','64','77'],
            palette=['coral','coral','coral','lightskyblue','lightskyblue','lightskyblue','lightskyblue','lightskyblue',
                  'firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick','firebrick',
                  'royalblue','royalblue','royalblue'])

In [ ]:
set(tdata.obs['celltype3_pub'])

In [ ]:
sc.set_figure_params()

In [ ]:
sc.pl.violin(tdata, keys='latent_time',groupby='leiden',
             order=['10','16','3','15','1','13','8','9','0','6','11','5'],
             save='-latent_time_per_cluster.tdata.svg')

In [ ]:
sc.pl.violin(tdata[tdata.obs['Sample type']=='TIL'], keys='latent_time',groupby='leiden',
             order=['10','16','3','15','1','13','8','9','0','6','11','5'],
            save='-latent_time_per_cluster.tdata_TIL.svg')

In [ ]:
scv.tl.paga(tdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')



In [ ]:
sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,10)

cr.pl.cluster_fates(tdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=5, edge_width_scale=1, max_edge_width=4, title='directed PAGA')


In [ ]:
sc.set_figure_params()

In [ ]:
cr.tl.lineage_drivers(rdata, use_raw=False, backward=False)


In [ ]:
cr.tl.lineage_drivers(subp, use_raw=False, backward=False)


In [ ]:
cr.pl.lineage_drivers(subp, lineage="5", n_genes=10)

In [ ]:

cr.pl.lineage_drivers(rdata, lineage="5", n_genes=10)

In [ ]:
cr.tl.lineage_drivers(tfdata, use_raw=False, backward=False)

cr.pl.lineage_drivers(tfdata, lineage="5", n_genes=10)

In [ ]:

cr.tl.lineage_drivers(nrnadjdata, use_raw=False, backward=False)

cr.pl.lineage_drivers(nrnadjdata, lineage="5", n_genes=10)

In [ ]:
cr.tl.lineage_drivers(nrbdata, use_raw=False, backward=False)
cr.tl.lineage_drivers(rbdata, use_raw=False, backward=False)


In [ ]:
cr.tl.lineage_drivers(tdata, use_raw=False, backward=False)

#cr.tl.lineage_drivers(subdata, use_raw=True, backward=False,cluster_key='leiden', clusters=['8','9','0','11'])

cr.pl.lineage_drivers(tdata, lineage="5", n_genes=10)

In [ ]:
scv.pl.velocity_embedding_stream(rdata, basis='umap',color=['Sample type'],
                                 min_mass=3.4)

In [ ]:
scv.pl.velocity_embedding_stream(nrnadjdata, basis='umap',color=['Sample type'],
                                 min_mass=3.4)

In [ ]:
scv.pl.velocity_embedding_stream(tdata, basis='umap',color=['Sample type'],
                                 min_mass=3.4, 
                                 save='-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-SampleType.tdata.svg')

In [ ]:
scv.pl.velocity_embedding_stream(tdata, basis='umap',color=['celltype3_pub'],legend_loc='right',
                                 min_mass=3.4, 
                                 save='-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-celltype.tdata.svg')

In [ ]:
root_idx = np.where(tdata.obs['initial_states'] == '10')[0][0]
tdata.uns['iroot'] = root_idx
sc.tl.dpt(tdata)



In [ ]:
scv.pl.scatter(tdata, color=['leiden', root_idx, 'latent_time', 'dpt_pseudotime'], fontsize=16,
               cmap='viridis', perc=[2, 98], colorbar=True, rescale_color=[0, 1],
               title=['leiden', 'root cell', 'latent time', 'dpt pseudotime'],
              save='-LatentTime-Pseudotime-CD8Tcell-All-PBMCandTIL-dyn-SampleType.tdata.svg')


In [ ]:
#model = cr.ul.models.GAM(subdata)
model = cr.ul.models.GAM(tdata)
cr.pl.gene_trends(tdata, model=model, data_key='X',
                  genes=['GZMB','CREM', 'XCL1','PDCD1','TOX','HAVCR2'], ncols=3,use_raw=True,
                  time_key='latent_time', same_plot=True, hide_cells=True,
                  figsize=(15, 4), n_test_points=200)


In [ ]:
#range(0,1,0.1)
mypart={}
mysplits=np.arange(0, 1, 0.05)

mysplits

In [ ]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [ ]:

def get_partition(rdata,myg,category):
    mypart=pd.DataFrame()
    mysplits=np.arange(0, 1, 0.05)
    for i in range(len(mysplits)-1):
        tmp=rdata[(rdata.obs['latent_time']>=mysplits[i])&(rdata.obs['latent_time']<mysplits[i+1])].raw[:,myg].X.todense().flatten().tolist()[0]
        tmp=pd.DataFrame([sum(x)/len(x) for x in chunkIt(tmp, min(len(tmp),20))])
        tmp['Lat']=mysplits[i+1]
        tmp['Cat']=category
        mypart=pd.concat([mypart,tmp])
    return(mypart)

In [ ]:
boldg=[ 'XCL1','XCL2','IFITM1','CD69','KLRB1','ZNF331','ITGAL',
       'LGALS9','DUSP1','DUSP4','AMICA1','ITGA1','KLRK1','KLRD1','HAVCR2','CD2',
                            'KLRC2','CXCR6','FAM3C','PRF1','TNFRSF1B',
                            'RDH10','TNFSF4','TIGIT','LAG3','CCL3','CCL4','CCL4L1','CCL4L2','GRN','NAMPT',
                            'TNFSF13B','HSPA6','HSPH1','SERPINH1','HSPA1A','HSPA1B']

boldgr=[ 'XCL1','XCL2','IFITM1','AMICA1','ITGA1','KLRK1','KLRD1','HAVCR2','CD2',
                            'KLRC2','CXCR6','FAM3C','PRF1','TNFRSF1B',
                            'RDH10','TNFSF4','TIGIT','LAG3']


boldgnr=[ 'CD69','KLRB1','ZNF331','ITGAL',
       'LGALS9','DUSP1','DUSP4','CCL3','CCL4','CCL4L1','CCL4L2','GRN','NAMPT',
                            'TNFSF13B','HSPA6','HSPH1','SERPINH1','HSPA1A','HSPA1B']

In [ ]:
myg='HSPA1A' #
myg='HSPA1B' #
myg='ATF3' #
myg='SERPINH1' #

#for myg in ['HSPA6','HSPH1','CCL4L1','CCL4L2','KLRC2','FAM3C','CCL4','PRF1','TNFRSF1b','RDH10','LAG3']:
for myg in ['KLRC2','RDH10','PRF1','CXCR6','XCL1','DUSP1','CD69','ZNF331']:
#for myg in boldg:
    
    #rpart=get_partition(rdata[rdata.obs['Sample type']=='TIL'],myg,'R')
    rpart=get_partition(rdata,myg,'R')

    tfpart=get_partition(tfdata,myg,'TF')
    nrpart=get_partition(nrdata,myg,'NR_adj')
    nrnadjpart=get_partition(nrnadjdata,myg,'NR_nadj')

    rpart.columns=[myg,'Latent_Time','RCat']
    tfpart.columns=[myg,'Latent_Time','RCat']
    nrpart.columns=[myg,'Latent_Time','RCat']
    nrnadjpart.columns=[myg,'Latent_Time','RCat']

    mypart=pd.concat([rpart,tfpart,nrpart,nrnadjpart])

    fig=sns.lineplot(data=mypart, y=myg, x="Latent_Time",hue="RCat")
    cxcr6=mypart.copy()
    fig.figure.savefig(figdir+'ExprPerTime-'+velosubset+'-'+mysubset+'-'+subcat+'-'+myg+'-per-response.subsettrajectorymap.svg')
    plt.clf()

In [ ]:
aha

In [ ]:
myg='HSPA1A' #
myg='HSPA1B' #
myg='ATF3' #
myg='SERPINH1' #

#for myg in ['HSPA6','HSPH1','CCL4L1','CCL4L2','KLRC2','FAM3C','CCL4','PRF1','TNFRSF1b','RDH10','LAG3']:
#for myg in boldg:
#for myg in ['DUSP4','KLRC2','RDH10','PRF1','CXCR6','XCL1','DUSP1','CD69','ZNF331']:
#for myg in ['DUSP4','LAG3','HSPH1','HSPA1A','HSPA1B']:
#for myg in ['AMICA1','SELL','DNAJA1','PPP1R15A','BTG1','TSC22D3','NAMPT','BTLA']:
for myg in ['XCL2','RDH10','IFITM1','ITGAL','CD2','GRN','TNFSP13B','SERPINH1']:
    #rpart=get_partition(rdata[rdata.obs['Sample type']=='TIL'],myg,'R')
    rbpart=get_partition(rbdata,myg,'R')

    nrbpart=get_partition(nrbdata,myg,'NR')

    rbpart.columns=[myg,'Latent_Time','RCat']
    nrbpart.columns=[myg,'Latent_Time','RCat']

    mypart=pd.concat([rbpart,nrbpart]).reset_index(drop=True)

    fig=sns.lineplot(data=mypart, y=myg, x="Latent_Time",hue="RCat")
    cxcr6=mypart.copy()
    #fig.figure.savefig(figdir+'ExprPerTime-'+velosubset+'-'+mysubset+'-'+subcat+'-'+myg+'-per-response-simple.subsettrajectorymap.pdf')
    fig.figure.savefig(figdir+'ExprPerTime-'+velosubset+'-'+mysubset+'-'+subcat+'-'+myg+'-per-response-simple.subsettrajectorymap.svg', format='svg', bbox_inches="tight", dpi=300)
    plt.clf()

In [ ]:
myg='HSPA1A' #
myg='HSPA1B' #
myg='ATF3' #
myg='SERPINH1' #

#for myg in ['HSPA6','HSPH1','CCL4L1','CCL4L2','KLRC2','FAM3C','CCL4','PRF1','TNFRSF1b','RDH10','LAG3']:
for myg in boldg:
    #rpart=get_partition(rdata[rdata.obs['Sample type']=='TIL'],myg,'R')
    rbpart=get_partition(rbdata_mature,myg,'R')

    nrbpart=get_partition(nrbdata_mature,myg,'NR')

    rbpart.columns=[myg,'Latent_Time','RCat']
    nrbpart.columns=[myg,'Latent_Time','RCat']

    mypart=pd.concat([rbpart,nrbpart])

    fig=sns.lineplot(data=mypart, y=myg, x="Latent_Time",hue="RCat")
    cxcr6=mypart.copy()
    fig.figure.savefig(figdir+'ExprPerTime-'+velosubset+'-'+mysubset+'-'+subcat+'-'+myg+'-per-response-simple-mature.subsettrajectorymap.pdf')
    plt.clf()

In [ ]:
set(tdata.obs['Respond'])

In [ ]:
tdata.obs['leidenRcat']=tdata.obs['leiden'].astype('str')+tdata.obs['RCat'].astype('str')
tdata.obs['leidenRespond']=tdata.obs['leiden'].astype('str')+tdata.obs['Respond'].astype('str')

In [ ]:
#sc.pl.matrixplot(tdata[tdata.obs['leiden'].isin(['1','9','11','0','5','6'])],
#                 var_names=boldg,
#                 groupby='leidenRcat',dendrogram=True,standard_scale='var',
#                svg='-boldg-c1-9-11-0-5-6.CD8Tvelo.leidenRespond.svg')

In [ ]:
sc.pl.matrixplot(tdata[tdata.obs['leiden'].isin(['1','9','11','0','5','6'])],
                 var_names=boldg,
                 groupby='leidenRespond',categories_order=['1R','1PD','9R','9PD','0R','0PD',
                                                           '11R','11PD','6R','6PD','5R','5PD'],standard_scale='var',
                 save='-boldg-c1-9-11-0-5-6.CD8Tvelo.leidenRespond.svg')

In [ ]:
sc.pl.matrixplot(tdata[tdata.obs['leiden'].isin(['1','9','11','0','5','6'])],
                 var_names=boldgr,
                 groupby='leidenRespond',categories_order=['1R','1PD','9R','9PD','0R','0PD',
                                                           '11R','11PD','6R','6PD','5R','5PD'],standard_scale='var')

In [ ]:
sc.pl.matrixplot(tdata[tdata.obs['leiden'].isin(['15','13','8','11','0','5','6'])],
                 var_names=boldg,
                 groupby='leidenRespond',categories_order=['15R','15PD','13R','13PD','8R','8PD','0R','0PD',
                                                           '11R','11PD','6R','6PD','5R','5PD'],standard_scale='var',
                      save='-boldg-c1-15-13-8-11-0-5-6.CD8Tvelo.leidenRespond.svg')

In [ ]:
sc.pl.matrixplot(tdata[tdata.obs['leiden'].isin(['1','9','11','0','5','6'])],
                 var_names=boldgnr,
                 groupby='leidenRespond',categories_order=['1R','1PD','9R','9PD','0R','0PD',
                                                           '11R','11PD','6R','6PD','5R','5PD'],standard_scale='var')

In [ ]:
sc.pl.dotplot(tdata[tdata.obs['leiden'].isin(['1','9','11','0','5','6'])],
                 var_names=boldg,
                 groupby='leidenRespond',categories_order=['1R','1PD','9R','9PD','0R','0PD',
                                                           '11R','11PD','6R','6PD','5R','5PD'])

In [ ]:
sns.lineplot(data=mypart, y=myg, x="Latent_Time",hue="RCat")
itm2a=mypart.copy()

In [ ]:
sns.lineplot(data=mypart, y=myg, x="Latent_Time",hue="RCat")
klf10=mypart.copy()

In [ ]:

mygenes=cr.pl.heatmap(tdata, model, 
              genes=subdatamstat.var['to 5'].sort_values(ascending=False).index[:50],
              show_absorption_probabilities=True,use_raw=True
              ,show_all_genes=True,cluster_genes=True,
              lineages="5", n_jobs=4, backend='loky',figsize=(10,15),return_genes=True)

In [ ]:
#lin_drivers_c8_9.sort_values(ascending=True)[0:40]

In [ ]:
#cr.tl.lineage_drivers(subdata, use_raw=True, backward=False,cluster_key='leiden', clusters=['8','9','0','11'])
cr.tl.lineage_drivers(rdata, use_raw=True, backward=False,cluster_key='leiden')
lin_drivers_r=rdata.raw.var['to 5']


cr.tl.lineage_drivers(nrdata, use_raw=True, backward=False,cluster_key='leiden')
lin_drivers_nr=nrdata.raw.var['to 5']

cr.tl.lineage_drivers(tfdata, use_raw=True, backward=False,cluster_key='leiden')
lin_drivers_tf=tfdata.raw.var['to 5']

cr.tl.lineage_drivers(nrnadjdata, use_raw=True, backward=False,cluster_key='leiden')
lin_drivers_nrnadj=nrnadjdata.raw.var['to 5']

cr.tl.lineage_drivers(rbdata, use_raw=True, backward=False,cluster_key='leiden')
lin_drivers_rb=rbdata.raw.var['to 5']

cr.tl.lineage_drivers(nrbdata, use_raw=True, backward=False,cluster_key='leiden')
lin_drivers_nrb=nrbdata.raw.var['to 5']


sc.set_figure_params()
cr.tl.lineage_drivers(tdata)

#


In [ ]:
set(lin_drivers_rb[np.abs(lin_drivers_rb)>0.1].index)-set(lin_drivers_nrb[np.abs(lin_drivers_nrb)>0.1].index)

In [ ]:
set(lin_drivers_nrb[np.abs(lin_drivers_nrb)>0.1].index)-set(lin_drivers_rb[np.abs(lin_drivers_rb)>0.1].index)

In [ ]:
choicen=set(lin_drivers_nr[np.abs(lin_drivers_nr)>0.1].index).intersection(set(lin_drivers_nrnadj[np.abs(lin_drivers_nrnadj)>0.1].index))-set(lin_drivers_tf[np.abs(lin_drivers_tf)>0.1].index)-set(lin_drivers_r[np.abs(lin_drivers_r)>0.1].index)

In [ ]:
lin_drivers_nr[np.abs(lin_drivers_nr)>0.1].sort_values(ascending=True)

In [ ]:
choicen

In [ ]:
choice=set(lin_drivers_r[np.abs(lin_drivers_r)>0.1].index).intersection(set(lin_drivers_tf[np.abs(lin_drivers_tf)>0.1].index))-set(lin_drivers_nrnadj[np.abs(lin_drivers_nrnadj)>0.1].index)-set(lin_drivers_nr[np.abs(lin_drivers_nr)>0.1].index)

In [ ]:
choice

In [ ]:
lin_drivers_r[np.abs(lin_drivers_r)>0.1].to_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_rdata_leiden_'+i+'.tsv',sep='\t')

In [ ]:
lin_drivers_r.sort_values(ascending=False)[0:100]

In [ ]:
lin_drivers_tf.sort_values(ascending=False)[0:100]

In [ ]:
lin_drivers_nr.sort_values(ascending=False)[0:100]

In [ ]:
lin_drivers_nrnadj.sort_values(ascending=False)[0:100]

In [ ]:
tdata.raw.var['to 5']

In [ ]:
lin_drivers=tdata.raw.var['to 5']


In [ ]:
DEgenes=bc.tl.dge.get_de(tdata,'leiden',demethod='wilcoxon',topnr=5000, logfc=1,padj=0.05)


In [ ]:
### Select only top genes (in order of p-val) for 2 clusters  plot expression per cluster
tops=list(DEgenes['8']['Name'][0:20])+list(DEgenes['9']['Name'][0:20])
sc.pl.dotplot(adata, var_names=tops,groupby='leiden',dendrogram=True)

In [ ]:
sc.pl.umap(tdata,color=list(lin_drivers_r.sort_values(ascending=True).index[0:8]))


In [ ]:
sc.pl.umap(tdata,color=list(lin_drivers_tf.sort_values(ascending=True).index[0:8]))


In [ ]:
sc.pl.umap(tdata,color=list(lin_drivers_nr.sort_values(ascending=False).index[0:8]))


In [ ]:
choice.intersection(DEgenes['8']['Name'])


In [ ]:
choice.intersection(DEgenes['15']['Name'])

In [ ]:
sc.pl.umap(rdata,color=['IRF8','LRRN3','ADTRP', 'DGKA','WDR41','WDR70','TFAM','AKT2'])


In [ ]:
sc.pl.umap(tfdata,color=list(lin_drivers_r[list(choice)].sort_values(ascending=True).index[0:8]))


In [ ]:
subdata.var['to 5'].sort_values(ascending=False).to_csv(figdir+'/CellRank_TopLineageGenes_'+velosubset+'-'+mysubset+'_'+subcat+'_tdata.tsv',sep='\t')


In [ ]:
list(subdata.var['to 5'].sort_values(ascending=False).index[0:20])


In [ ]:
list(subdata.var['to 5'].sort_values(ascending=True).index[0:20])


In [ ]:
cr.pl.lineage_drivers(subdata, lineage="5", n_genes=10)


In [ ]:
rdata.obs['leidenvar']=rdata.obs['leiden']

In [ ]:
rdata.obs['leidenvar']=rdata.obs['leidenvar'].replace('0','8')
rdata.obs['leidenvar']=rdata.obs['leidenvar'].replace('13','15')
rdata.obs['leidenvar']=rdata.obs['leidenvar'].replace('16','10')
rdata.obs['leidenvar']=rdata.obs['leidenvar'].replace('11','5')

In [ ]:
scv.tl.rank_dynamical_genes(rdata, groupby='leidenvar')
df = scv.get_df(rdata, 'rank_dynamical_genes/names')
df.head(10)


In [ ]:
scv.tl.rank_dynamical_genes(tdata, groupby='leiden')


In [ ]:
df = scv.get_df(tdata, 'rank_dynamical_genes/names')
df.head(10)


In [ ]:
df.head(50).to_csv(figdir+'/TopDynamicGenesPerCluster_'+velosubset+'-'+mysubset+'_'+subcat+'_loom-dyn_Top50Ranked.tdata.tsv',sep='\t')


In [ ]:
cdata=tdata.copy()

In [ ]:
transitiongenes1=cdata.raw.var['to 5'][list(df.head(50)['10'])][np.abs(cdata.raw.var['to 5'][list(df.head(50)['10'])])>0.01].sort_values()
transitiongenes2=cdata.raw.var['to 5'][list(df.head(50)['16'])][np.abs(cdata.raw.var['to 5'][list(df.head(50)['16'])])>0.01].sort_values()
initialgenes=list(set(transitiongenes1.index).union(set(transitiongenes2.index)))

transitiongenes1=cdata.raw.var['to 5'][list(df.head(50)['3'])][np.abs(cdata.raw.var['to 5'][list(df.head(50)['3'])])>0.01].sort_values()
transitiongenes2=cdata.raw.var['to 5'][list(df.head(50)['1'])][np.abs(cdata.raw.var['to 5'][list(df.head(50)['1'])])>0.01].sort_values()
transitiongenes=list(set(transitiongenes1.index).union(set(transitiongenes2.index)))

tamcxgenes1=cdata.raw.var['to 5'][list(df.head(50)['8'])][np.abs(cdata.raw.var['to 5'][list(df.head(50)['8'])])>0.01].sort_values()
tamcxgenes2=cdata.raw.var['to 5'][list(df.head(50)['9'])][np.abs(cdata.raw.var['to 5'][list(df.head(50)['9'])])>0.01].sort_values()
tamcxgenes=list(set(tamcxgenes1.index).union(set(tamcxgenes2.index)))

tamcxgenes1=cdata.raw.var['to 5'][list(df.head(50)['13'])][np.abs(cdata.raw.var['to 5'][list(df.head(50)['13'])])>0.01].sort_values()
tamcxgenes2=cdata.raw.var['to 5'][list(df.head(50)['15'])][np.abs(cdata.raw.var['to 5'][list(df.head(50)['15'])])>0.01].sort_values()
ccgenes=list(set(tamcxgenes1.index).union(set(tamcxgenes2.index)))

tamcxgenes1=cdata.raw.var['to 5'][list(df.head(50)['11'])][np.abs(cdata.raw.var['to 5'][list(df.head(50)['11'])])>0.01].sort_values()
tamcxgenes2=cdata.raw.var['to 5'][list(df.head(50)['5'])][np.abs(cdata.raw.var['to 5'][list(df.head(50)['5'])])>0.01].sort_values()
tamcxgenes3=cdata.raw.var['to 5'][list(df.head(50)['6'])][np.abs(cdata.raw.var['to 5'][list(df.head(50)['6'])])>0.01].sort_values()
exhgenes=list(set(tamcxgenes1.index).union(set(tamcxgenes2.index)).union(set(tamcxgenes3.index)))


In [ ]:
aall=list(set(transitiongenes).union(set(initialgenes)).union(set(tamcxgenes)).union(set(ccgenes)).union(set(exhgenes)))

In [ ]:
set(aall)

In [ ]:
#set(tamcxgenes)

In [ ]:
#set(initialgenes).union(set(transitiongenes)).union(set(tamcxgenes))-set(ccgenes)-set(exhgenes)

In [ ]:
a1=list(set(initialgenes).union(set(transitiongenes))-set(ccgenes)-set(exhgenes))
#union(set(tamcxgenes))
a2=list(set(ccgenes)-set(exhgenes)-set(initialgenes))
a3=list(set(exhgenes)-set(initialgenes).union(set(transitiongenes)))


In [ ]:
a3

In [ ]:
#cdata.var['to 5'].sort_values(ascending=True)[0:30]
scv.pl.heatmap(cdata, var_names=aall, sortby='latent_time', 
               col_color='leiden', n_convolve=30,show=True,
               row_cluster=True,
               font_scale=0.6,  figsize=(6, 12))


In [ ]:
#cdata.var['to 5'].sort_values(ascending=True)[0:30]
scv.pl.heatmap(cdata, var_names=a2, sortby='latent_time', 
               col_color='leiden', n_convolve=30,show=True,
               row_cluster=True,
               font_scale=0.6,  figsize=(6, 7))


In [ ]:
#cdata.var['to 5'].sort_values(ascending=True)[0:30]
scv.pl.heatmap(cdata, var_names=a1, sortby='latent_time', 
               col_color='leiden', n_convolve=30,show=True,
               row_cluster=True,
               font_scale=0.6,  figsize=(6, 7))


In [ ]:
ssdata=tdata[tdata.obs['Sample type']=='TIL'].copy()
#ssdata=tdata[tdata.obs['leiden'].isin(['8','9','15'])].copy()

#=bc.tl.dge.get_de(subdataa,'RCat',demethod='wilcoxon',topnr=5000, logfc=np.log(1.5),padj=0.1)
### try on individual tissues separately
b1DE=bc.tl.dge.get_de(ssdata[ssdata.obs['Lesion'].isin(['LN'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
b2DE=bc.tl.dge.get_de(ssdata[ssdata.obs['Lesion'].isin(['Brain'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
b3DE=bc.tl.dge.get_de(ssdata[ssdata.obs['Lesion'].isin(['Lung','Sinon','Subc'])].copy(),'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)

allDE1=bc.tl.dge.get_de(ssdata[ssdata.obs['Response3'].isin(['R','NR'])],'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
allDE2=bc.tl.dge.get_de(ssdata[ssdata.obs['Response3'].isin(['TF','PD'])],'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)
allDE=bc.tl.dge.get_de(ssdata,'Respond',topnr=5000, logfc=np.log(1.5),padj=0.1)



In [ ]:
### Intersection of all combinations
strDE=set(b1DE['PD']['Name']).intersection(set(allDE['PD']['Name'])).intersection((set(b2DE['PD']['Name'])).intersection(set(b3DE['PD']['Name']))).intersection((set(allDE1['PD']['Name'])).intersection(set(allDE2['PD']['Name'])))
strDE2=set(b1DE['R']['Name']).intersection(set(allDE['R']['Name'])).intersection((set(b2DE['R']['Name'])).intersection(set(b3DE['R']['Name']))).intersection((set(allDE1['R']['Name'])).intersection(set(allDE2['R']['Name'])))



In [ ]:
strDE

In [ ]:
strDE2

In [ ]:
### Complete comparison + LN only + Brain | Others + RvsNR | PDvsTF
strDE=set(b1DE['PD']['Name']).intersection(set(allDE['PD']['Name'])).intersection((set(b2DE['PD']['Name'])).union(set(b3DE['PD']['Name']))).intersection((set(allDE1['PD']['Name'])).union(set(allDE2['PD']['Name'])))
strDE2=set(b1DE['R']['Name']).intersection(set(allDE['R']['Name'])).intersection((set(b2DE['R']['Name'])).union(set(b3DE['R']['Name']))).intersection((set(allDE1['R']['Name'])).union(set(allDE2['R']['Name'])))



In [ ]:
strDE

In [ ]:
strDE2

In [ ]:
sc.pl.umap(cdata,color=list(cdata.var['to 5'][transitiongenes].sort_values().index))


In [ ]:
tmp=cdata.var['to 5'][list(set(tamcxgenes)-set(transitiongenes)-set(exhgenes))].sort_values()
tmp

In [ ]:
sc.pl.umap(cdata,color=list(tmp.sort_values().index))


In [ ]:
scv.pl.velocity(cdata, ['CRTAM', 'IL7R','PLAUR'], ncols=2, add_outline=True)


In [ ]:
ofinterest=set([item for sublist in df.head(50).values.tolist() for item in sublist])
ofinterest=cdata.var['to 5'][list(ofinterest)][np.abs(cdata.var['to 5'][list(ofinterest)])>0.1]


In [ ]:
ofinterest.sort_values()

In [ ]:
velogenes=cdata.var.loc[(adata.var['velocity_genes']==True)&(cdata.var['fit_likelihood']>.1),:]


In [ ]:
list(set(velogenes.index))

In [ ]:
len(set(ofinterest.index)-set(velogenes.index))/len(set(ofinterest)) ## How many are specific 


In [ ]:
ofinterest=ofinterest[list(set(ofinterest.index).intersection(set(velogenes.index)))].sort_values()


In [ ]:
top_genes = cdata.var['fit_likelihood'].sort_values(ascending=False).index#[:300]
top_genes1=cdata.var['fit_likelihood'].sort_values(ascending=False).index[:250]


In [ ]:
invelo=[]
for x in list(top_genes):
    invelo=invelo+list(set([x]).intersection(set(velogenes.index)))


scv.pl.heatmap(cdata, var_names=list(ofinterest.index), sortby='latent_time', 
               col_color='leiden', n_convolve=30,show=True,
               font_scale=0.6,  figsize=(6, 7))


In [ ]:
scv.pl.heatmap(cdata, var_names=list(ofinterest.index), sortby='latent_time', 
               col_color='celltype3_pub', n_convolve=30,show=True,row_cluster=True,
               font_scale=0.6,  figsize=(6, 7))



In [ ]:
sc.pl.umap(cdata,color=list(ofinterest.index))

In [ ]:
#ddata=subdata.copy()
ddata=tdata.copy()

In [ ]:
sc.pl.umap(ddata,color=['latent_time','leiden'])

In [ ]:
#ddata=ddata[ddata.obs['Sample type'].isin(['TIL'])].copy()
#ddata=ddata[ddata.obs['leiden'].isin(['9','8','0','5','6','4','11','2','15','13','12','14'])].copy()
#ddata=ddata[ddata.obs['celltype3_pub'].isin(['cytokine secreting effector CD8-positive T cell',
#                                             'exhausted-like CD8-positive T cell'])].copy()

In [ ]:
#set(adata.obs['celltype3_pub'])

In [ ]:
sc.pl.umap(ddata,color=['latent_time','leiden'])

In [ ]:
sc.pl.umap(ddata,color=['leiden'], legend_loc='on data')

In [ ]:
exh=['LAG3','CD38','PDCD1','ENTPD1','ITGAE','HAVCR2','CTLA4','TOX',
     'LAYN','IFNG','PRF1','GNLY','XCL1','XCL2','GZMM','KLRB1','ITGB1', 'ARL4C','IL7R','CCR7','SELL','TCF7',]
sc.pl.dotplot(ddata,var_names=exh,groupby='celltype3_pub')

In [ ]:
sc.pl.dotplot(ddata,var_names=exh,groupby='leiden', dendrogram=True)

In [ ]:
ddata=tdata.copy()
#ddata=ddata[ddata.obs['Sample type'].isin(['TIL'])].copy()
#ddata=ddata[ddata.obs['Sample type'].isin(['TIL'])].copy()
#ddata=ddata[ddata.obs['Sample type'].isin(['TIL'])].copy()


In [ ]:
ddata.obs['R_clus']=ddata.obs['RCat'].astype(str)+'-'+[x.replace("-", "_") for x in ddata.obs['leiden'].astype(str)]
ddata.obs['Pat_clus']=ddata.obs['PatientID'].astype(str)+'-'+[x.replace("-", "_") for x in ddata.obs['leiden'].astype(str)]
dfPat = ddata.obs.groupby('Pat_clus')['velocity_length', 'velocity_confidence','latent_time'].mean().T

In [ ]:
dfPat.to_csv(figdir+'Velocity-per_patient'+velosubset+'-'+mysubset+'-'+subcat+'-newest.tsv',sep='\t')

In [ ]:
dfPat

In [ ]:
Resp=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0] for x in list(dfPat.columns)]
PatientID=[x.split('-')[0]  for x in list(dfPat.columns)]
ClusterID=[x.split('-')[1]  for x in list(dfPat.columns)]

In [ ]:
mydf=pd.DataFrame([list(dfPat.loc['velocity_length',:]),Resp,PatientID,ClusterID]).transpose()
mydf.index=dfPat.columns
mydf.columns=['velocity_length','Response','PatientID','Cluster']

mydfl=pd.DataFrame([list(dfPat.loc['latent_time',:]),Resp,PatientID,ClusterID]).transpose()
mydfl.index=dfPat.columns
mydfl.columns=['latent_time','Response','PatientID','Cluster']

In [ ]:
mydf.groupby('PatientID', as_index=False)['velocity_length']

In [ ]:
color_dict = {'R': 'coral', 'TF': 'firebrick', 'NR_nadj': 'lightskyblue','NR_adj': 'royalblue'}

#### Velocity test first

In [ ]:
#corder=['8','6','1','21']
mydf['velocity_length']=mydf['velocity_length'].astype('float')
velMean=mydf.groupby('PatientID')['velocity_length'].mean()
patid=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0]  for x in list(velMean.index)]
totest=pd.DataFrame([list(velMean),patid]).transpose()
totest.columns=['Val','Cat']

In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='R',:]['Val']),list(totest.loc[totest['Cat']=='NR_nadj',:]['Val']))

In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='TF',:]['Val']),list(totest.loc[totest['Cat']=='NR_adj',:]['Val']))

In [ ]:
set(mydf['Response'])

In [ ]:
totestk=totest.copy()

In [ ]:
fig=sns.boxplot(y='Val',x='Cat',data=totestk,palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
fig=sns.swarmplot(x='Cat',y='Val',data=totestk,color='black',order=['R','TF','NR_nadj','NR_adj'])
#fig.figure.savefig(figdir+'Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap.pdf')
#fig.figure.savefig(figdir+'Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
fig=sns.boxplot(y='Val',x='Cat',data=totestk,palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
fig=sns.swarmplot(x='Cat',y='Val',data=totestk,color='black',order=['R','TF','NR_nadj','NR_adj'])
#fig.figure.savefig(figdir+'Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap.pdf')
fig.figure.savefig(figdir+'Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
totest=list(itertools.combinations(['R','TF','NR_nadj','NR_adj'], 2))
pwilc={}
pt={}
for pairs in totest:
    pwilc[pairs[0]+'-'+pairs[1]]=stats.mannwhitneyu(list(totestk.loc[totestk['Cat']==pairs[0],:]['Val']), 
                           list(totestk.loc[totestk['Cat']==pairs[1],:]['Val']))[1]
    pt[pairs[0]+'-'+pairs[1]]=stats.ttest_ind(list(totestk.loc[totestk['Cat']==pairs[0],:]['Val']), 
                           list(totestk.loc[totestk['Cat']==pairs[1],:]['Val']))[1]

myps=pd.DataFrame([pwilc,pt]).transpose()
myps.columns=['MannWhitney','T-test']

#myps.to_csv(figdir+'Pval-Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap.csv',sep='\t')

In [ ]:
myps

In [ ]:
corder=list(set(mydf['Cluster']))

In [ ]:
#corder=['15','14','1','9','8','0','11','6','5']
for i in corder:
    sns.boxplot(y='velocity_length',x='Response',data=mydf.loc[mydf['Cluster']==i,:],
                palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
    sns.swarmplot(x='Response',y='velocity_length',data=mydf.loc[mydf['Cluster']==i,:],
                  color='black',order=['R','TF','NR_nadj','NR_adj'])
    print('Cluster '+i)
    plt.figure()

#### Latent time next

In [ ]:
mydfl

In [ ]:
#corder=['8','6','1','21']
mydfl['latent_time']=mydfl['latent_time'].astype('float')
velMean=mydfl.groupby('PatientID')['latent_time'].mean()
patid=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0]  for x in list(velMean.index)]
totest=pd.DataFrame([list(velMean),patid]).transpose()
totest.columns=['Val','Cat']

ss.mannwhitneyu(list(totest.loc[totest['Cat']=='R',:]['Val']),list(totest.loc[totest['Cat']=='NR_nadj',:]['Val']))



In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='TF',:]['Val']),list(totest.loc[totest['Cat']=='NR_adj',:]['Val']))

In [ ]:
totestk=totest.copy()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (3,4)

In [ ]:
fig=sns.boxplot(y='Val',x='Cat',data=totestk,palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
fig=sns.swarmplot(x='Cat',y='Val',data=totestk,color='black',order=['R','TF','NR_nadj','NR_adj'])
#fig.figure.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap-allTILs.svg')


In [ ]:
fig=sns.boxplot(y='Val',x='Cat',data=totestk,palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
fig=sns.swarmplot(x='Cat',y='Val',data=totestk,color='black',order=['R','TF','NR_nadj','NR_adj'])
#fig.figure.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap-allTILs.svg')
fig.figure.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
#sc.pl.umap(ddata, color=['latent_time', 'dpt_pseudotime'])
sc.pl.umap(ddata, color=['latent_time'])

In [ ]:
i='5'
sns.boxplot(y='latent_time',x='Response',data=mydfl.loc[mydfl['Cluster'].isin([i]),:],
                palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
sns.swarmplot(x='Response',y='latent_time',data=mydfl.loc[mydfl['Cluster'].isin([i]),:],
                  color='black',order=['R','TF','NR_nadj','NR_adj']).set_title('Cluster ' + i)


In [ ]:
### Cluster 9 p=0.018; 8 0.018; 11 0.037; 0 0.068
### Cluster 9 0.035; 8 0.035; 11 0.07; 0 0.14; 6 0.39; 5 0.14
tmp=mydfl.loc[mydfl['Cluster'].isin([i]),:].copy()
a1=ss.mannwhitneyu(list(tmp.loc[tmp['Response']=='R',:]['latent_time']),list(tmp.loc[tmp['Response']=='NR_nadj',:]['latent_time']))


In [ ]:
a2=ss.mannwhitneyu(list(tmp.loc[tmp['Response']=='TF',:]['latent_time']),list(tmp.loc[tmp['Response']=='NR_adj',:]['latent_time']))


In [ ]:
pvals=pd.Series([a1[1], a2[1]])
pvals.index=['R-NR','TF-NR']
pvals

In [ ]:
corder=list(set(mydf['Cluster']))
pvalslist={}
for i in corder:
    fig=sns.boxplot(y='latent_time',x='Response',data=mydfl.loc[mydf['Cluster']==i,:],
                palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
    fig=sns.swarmplot(x='Response',y='latent_time',data=mydfl.loc[mydfl['Cluster']==i,:],
                  color='black',order=['R','TF','NR_nadj','NR_adj']).set_title('Cluster '+i)
    
    tmp=mydfl.loc[mydfl['Cluster'].isin([i]),:].copy()
    a1=ss.mannwhitneyu(list(tmp.loc[tmp['Response']=='R',:]['latent_time']),list(tmp.loc[tmp['Response']=='NR_nadj',:]['latent_time']))
    a2=ss.mannwhitneyu(list(tmp.loc[tmp['Response']=='TF',:]['latent_time']),list(tmp.loc[tmp['Response']=='NR_adj',:]['latent_time']))
    pvals=pd.Series([a1[1], a2[1]])
    pvals.index=['R-NR','TF-NR']
    pvalslist['Cluster_'+i]=pvals

    print('Cluster '+i)
    fig.figure.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap.cluster-'+i+'.svg', format='svg', bbox_inches="tight", dpi=300)
    plt.figure()

In [ ]:
pd.DataFrame(pvalslist)

In [ ]:
pd.DataFrame(pvalslist).to_csv(figdir+'LatentTimePerClusterPatient-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.pvals.subsettrajectorymap.csv',sep='\t')

In [ ]:
figdir

In [ ]:
mydfl.to_csv(figdir+'LatentTimePerClusterPatient-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap.csv',sep='\t')

In [ ]:
totestk

In [ ]:

totestk.to_csv(figdir+'LatentTimePerPatient-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.subsettrajectorymap.csv',sep='\t')

In [ ]:
plt.figure(figsize=(4,3))
mydat=ddata.obs.loc[:,['RCat','latent_time']]
fig=sns.violinplot(x='RCat',y='latent_time',data=mydat, palette=color_dict, order=['R','TF','NR_nadj','NR_adj'])
fig.figure.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response-violins.svg') 


In [ ]:
lt={}
for resp in  list(color_dict.keys()):
    lt[resp]=ddata[ddata.obs['RCat']==resp].obs['latent_time']
mydat=ddata.obs.loc[:,['RCat','latent_time']]

plt.figure(figsize=(4,3))
myc=list(color_dict.values())
i=0
# Iterate through the five airlines
for resp in list(color_dict.keys()):
    subset = lt[resp]
    sns.distplot(subset, hist = False, kde = True,
                 kde_kws = {'linewidth': 3},
                 label = resp,color=myc[i])
    i=i+1

# Plot formatting
plt.legend(prop={'size': 10}, title = 'Responders')
plt.title('Density Plot across Responses')
plt.xlabel('Latent time')
plt.ylabel('Density')
#plt.savefig(outpath+'/LatentTime_RvsNR_CD8TNK_CCtoExh_TILonly.pdf')
#plt.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response-desity.pdf')
plt.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'.density.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
! jupyter nbconvert --to html scvelo-CD8T.ipynb